In [3]:
import numpy as np
import os
import pandas as pd
import pickle
from tqdm.notebook import tqdm # 노트북용 tqdm
import torch
import torch.nn.functional as F
from collections import defaultdict

# --- 기본 설정값 (원본 코드에서 가져옴) ---
NUM_AGENTS = 23
NUM_TEAM_PLAYERS = 11
H, W = 68, 104
# feature_cols 리스트는 실제 사용하는 것으로 채워야 합니다.
FEATURE_COLS = [
    'x', 'y', 'vx', 'vy', 'v', 'ax', 'ay', 'a',
    'is_teammate', 'is_goalkeeper',
    'distance_to_goal', 'sin_angle_to_goal', 'cos_angle_to_goal',
    'distance_to_ball', 'sin_angle_to_ball', 'cos_angle_to_ball',
    'cos_velocity_angle', 'sin_velocity_angle',
    'type_id'
]
COLS_TO_FLIP = ['x', 'y', 'vx', 'vy', 'ax', 'ay']

# --- 테스트 경로 설정 ---
# 이 부분은 실제 환경에 맞게 수정해주세요.
DATA_PATH = "/home/express-v2/data/processed" 
TEST_MATCH_ID = "126285" # 테스트하고 싶은 실제 경기 ID

In [4]:

def _merge_tracking_pressing_df(tracking_df, pressing_df, teams_df):
    """Merge tracking data with pressing intensity data and team information.
    
    This function combines tracking data, pressing intensity data, and team information
    into a single dataframe. It filters for alive ball states, normalizes timestamps,
    and handles ball identification.
    
    Args:
        tracking_df (pd.DataFrame): Tracking data containing player and ball positions.
        pressing_df (pd.DataFrame): Pressing intensity data for each frame.
        teams_df (pd.DataFrame): Team information containing player codes.
        
    Returns:
        pd.DataFrame: Merged dataframe with tracking, pressing, and team data.
        
    Raises:
        ValueError: If unexpected IDs are found where player_code is NaN.
    """
    # Merge tracking and pressing data
    total_df = pd.merge(
        tracking_df, pressing_df, 
        on=['game_id', 'period_id', 'timestamp', 'frame_id'], 
        how='left'
    )
    
    # Filter for alive ball states only
    total_df = total_df[total_df['ball_state'] == 'alive']
    
    # Normalize second half timestamps
    total_df.loc[total_df['period_id'] == 2, 'timestamp'] -= pd.Timedelta(minutes=45)
    
    # Merge with team information
    total_df = total_df.merge(
        teams_df[['pID', 'player_code']],
        how='left',
        left_on='id',
        right_on='pID'
    )
    total_df.drop(['pID'], axis=1, inplace=True)

    # Handle ball identification
    nan_mask = pd.isna(total_df['player_code'])
    nan_ids = set(total_df.loc[nan_mask, 'id'].unique())
    expected_ids_ball = {'ball'}
    expected_ids_empty = set()

    if not (nan_ids == expected_ids_ball or nan_ids == expected_ids_empty):
        raise ValueError(
            f"Found unexpected IDs in rows where player_code is NaN. "
            f"Expected: {{'ball'}} or {{}}, but found: {nan_ids}"
        )
    
    # Set ball ID as player_code for ball rows
    total_df.loc[nan_mask, 'player_code'] = total_df.loc[nan_mask, 'id']
    return total_df


In [5]:
def _preprocess_event_df(event_df, teams_df):
    """Preprocess event dataframe for analysis.
    
    This function normalizes time data, converts player IDs to consistent format,
    creates player codes, and merges team information with event data.
    
    Args:
        event_df (pd.DataFrame): Raw event dataframe containing match events.
        teams_df (pd.DataFrame): Team information dataframe containing player details.
        
    Returns:
        pd.DataFrame: Preprocessed event dataframe with normalized time data,
            consistent player IDs, and merged team information.
    """
    # Normalize time data to 0.04 second intervals
    event_df['time_seconds'] = (event_df['time_seconds'] / 0.04).round() * 0.04
    event_df['relative_time_seconds'] = (event_df['relative_time_seconds'] / 0.04).round() * 0.04
    
    # Convert to timedelta objects
    event_df['time_seconds'] = pd.to_timedelta(event_df['time_seconds'], unit='s')
    event_df['relative_time_seconds'] = pd.to_timedelta(event_df['relative_time_seconds'], unit='s')
    
    # Convert player_id to consistent string format
    event_df['player_id'] = event_df['player_id'].astype(int).astype(str)
    
    # Reset team dataframe index and create player codes
    teams_df.reset_index(drop=True, inplace=True)
    teams_df['player_code'] = teams_df.apply(
        lambda row: row['team'][0] + str(row['xID']).zfill(2), axis=1
    )

    # Merge event data with team information
    event_df = event_df.merge(
        teams_df,
        how='left',
        left_on='player_id',
        right_on='pID'
    )

    return event_df

In [6]:

def _normalize_coordinate_direction(df, home_team_id):
    """Normalize coordinate system to ensure consistent attack direction.
    
    This function normalizes the coordinate system within the DataFrame to ensure
    the home team always attacks in a consistent direction (left-to-right).
    It handles both period-based flipping and initial orientation correction.
    
    Args:
        df (pd.DataFrame): The tracking DataFrame to process. Must contain
            'period_id', 'frame_id', 'x', 'y', 'team_id' columns. Will also
            flip 'vx', 'vy', 'ax', 'ay' if they exist.
        home_team_id (str): The team ID of the home team.
        
    Returns:
        pd.DataFrame: A DataFrame with normalized coordinate directions.
            Does not modify the input df directly, works on a copy.
            
    Raises:
        ValueError: If minimum x value cannot be determined for orientation check.
    """
    df_normalized = df.copy()
    
    # Step 1: Unify direction for the second half
    second_half_mask = df_normalized['period_id'] == 2
    
    # Flip coordinates and related vectors for second-half data
    cols_to_flip = ['x', 'y', 'vx', 'vy', 'ax', 'ay']
    for col in cols_to_flip:
        if col in df_normalized.columns and df_normalized[col].dtype != 'object':
            df_normalized.loc[second_half_mask, col] = -df_normalized.loc[second_half_mask, col]
    
    # Step 2: Check home team attack direction and flip all if necessary
    first_period_frames = df_normalized[df_normalized['period_id'] == 1]['frame_id']
    if not first_period_frames.empty:
        first_frame_idx = first_period_frames.unique()[0]
        first_frame_df = df_normalized[
            (df_normalized['period_id'] == 1) & 
            (df_normalized['frame_id'] == first_frame_idx) & 
            (df_normalized['team_id'] != 'ball')
        ].copy()

        if not first_frame_df.empty and not first_frame_df['x'].isna().all():
            try:
                min_x_team_id = first_frame_df.loc[first_frame_df['x'].idxmin(), 'team_id']
                
                # If leftmost entity is not home team, flip all coordinates
                if min_x_team_id != home_team_id:
                    for col in cols_to_flip:
                        if col in df_normalized.columns and df_normalized[col].dtype != 'object':
                            df_normalized.loc[:, col] = -df_normalized.loc[:, col]
            except ValueError:
                print(f"Warning: Could not determine minimum 'x' value for frame {first_frame_idx}. Skipping orientation check.")
        else:
            print(f"Warning: No valid data found for the first frame ({first_frame_idx}) of period 1. Skipping main orientation check.")
    else:
        print("Warning: No data found for period 1. Skipping main orientation check.")
        
    return df_normalized

In [7]:
def _get_in_game_feature(df):
    # 1 time_diff 구하기
    time_diffs = []
    for _, row in df.iterrows():
        current_time = row['time_seconds']
        current_team = row['tID']

        # 상대 팀의 과거 이벤트만 필터링
        opponent_events = df[
            (df['tID'] != current_team) &
            (df['time_seconds'] < current_time)
        ]

        if not opponent_events.empty:
            last_opponent_time = opponent_events['time_seconds'].max()
            time_diff = (current_time - last_opponent_time).total_seconds()
        else:
            time_diff = 0.0

        time_diffs.append(time_diff)

    # 계산된 리스트를 새 컬럼으로 추가
    df['time_diff'] = time_diffs

    # 2. Goal diff 구하기

    # 골 여부 판별
    df['is_goal'] = (df['type_name'] == "shot") & (df['result_name'] == "Goal")

    # 누적 골 초기화
    att_goal_count = np.zeros(len(df), dtype=int)
    def_goal_count = np.zeros(len(df), dtype=int)

    for team in df['tID'].unique():
        # 공격/수비 여부 판단
        is_att = df['tID'] == team
        is_def = ~is_att

        # 공격팀 입장에서 자기가 골 넣은 경우
        att_goal = df['is_goal'] & is_att
        att_goal_cumsum = att_goal.cumsum().shift(fill_value=0)
        att_goal_count[is_att] = att_goal_cumsum[is_att]

        # 공격팀 입장에서 상대가 골 넣은 경우
        def_goal = df['is_goal'] & is_def
        def_goal_cumsum = def_goal.cumsum().shift(fill_value=0)
        def_goal_count[is_att] = def_goal_cumsum[is_att]
    df['att_goal_count'] = att_goal_count
    df['def_goal_count'] = def_goal_count
    df.drop(columns='is_goal', inplace=True)
    return df


In [8]:
# --- 단일 경기 데이터 로딩 ---
print(f"Loading data for match_id: {TEST_MATCH_ID}")

# 1. processed_dict 로딩
with open(f"{DATA_PATH}/{TEST_MATCH_ID}/{TEST_MATCH_ID}_processed_dict.pkl", "rb") as f:
    match_dict = pickle.load(f)

tracking_df = match_dict['tracking_df'].copy()
teams_dict = match_dict['teams'].copy()
home_team = teams_dict['Home'].copy()
away_team = teams_dict['Away'].copy()
teams_df = pd.concat([home_team, away_team])
meta_data = match_dict['meta_data']
# 2. pressing_intensity 로딩
with open(f"{DATA_PATH}/{TEST_MATCH_ID}/{TEST_MATCH_ID}_pressing_intensity.pkl", "rb") as f:
    pressing_df = pickle.load(f)

# 3. event_df 로딩
event_df = pd.read_csv(f"{DATA_PATH}/{TEST_MATCH_ID}/valid_events_filtered2.csv")
event_df = _preprocess_event_df(event_df, teams_df) # 위에서 정의한 헬퍼 함수 사용
event_df = _get_in_game_feature(event_df)
# 4. 데이터 통합
total_df = _merge_tracking_pressing_df(tracking_df, pressing_df, teams_df) # 위에서 정의한 헬퍼 함수 사용
tracking_df = _normalize_coordinate_direction(tracking_df, teams_dict['Home']['tID'].iloc[0]) # 위에서 정의한 헬퍼 함수 사용


print("Data loading complete!")

Loading data for match_id: 126285


/opt/conda/envs/press/lib/python3.11/site-packages/pandas/core/arrays/timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
/opt/conda/envs/press/lib/python3.11/site-packages/pandas/core/arrays/timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


Data loading complete!


In [9]:
total_dict = {TEST_MATCH_ID: {}}
total_dict

{'126285': {}}

In [10]:
def _check_pressing_success( row, event_df, teams_dict):
    """Check if pressing was successful based on subsequent events.
    
    This function analyzes events that occur within 5 seconds after a pressing
    situation to determine if the pressing team successfully gained possession.
    
    Args:
        row (pd.Series): Row containing pressing situation information.
        event_df (pd.DataFrame): Event dataframe containing match events.
        teams_dict (dict): Dictionary containing team information.
        
    Returns:
        bool: True if pressing was successful (possession gained), False otherwise.
    """
    # Events that indicate successful possession gain
    possession_gained_events = [
        'pass', 'dribble', 'recovery', 'interception', 'cross', 'throw_in', 
        'take_on', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick'
    ] 
    
    # Determine pressing team based on ball carrier's team
    if row['team_id'] == teams_dict['Home']['tID'].unique()[0]:
        pressing_team = teams_dict['Away']['tID'].unique()[0]
    elif row['team_id'] == teams_dict['Away']['tID'].unique()[0]:
        pressing_team = teams_dict['Home']['tID'].unique()[0]
    else:
        return False

    # Check events within 5 seconds after pressing
    check_timegap = pd.Timedelta(seconds=5)
    window_events = event_df[
        (event_df['period_id'] == row['period_id']) &
        (event_df['time_seconds'] >= row['timestamp']) &
        (event_df['time_seconds'] <= row['timestamp'] + check_timegap)
    ]
    event_teams = window_events['tID'].unique()

    if pressing_team in event_teams:
        pressing_team_events = window_events[window_events['tID'] == pressing_team]
        for _, event_row in pressing_team_events.iterrows():
            if event_row['type_name'] in possession_gained_events:
                # Always successful events
                if event_row['type_name'] in ['dribble', 'recovery', 'interception', 'cross', 
                                                'throw_in', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick']:
                    return True
                # Events that need success check
                elif event_row['type_name'] in ['pass', 'take_on']:
                    if event_row['result_name'] == 'Successful':
                        return True
        return False
    else:
        return False

In [11]:
def _generate_features(frame_df: pd.DataFrame) -> pd.DataFrame:
    """Generate additional features from tracking data for a single frame.
    
    This function calculates various features including binary attributes,
    distance and angle features relative to goal and ball, and velocity
    angle features for each player in the frame.
    
    Args:
        frame_df (pd.DataFrame): Player and ball data for a single frame.
            Must contain columns: 'id', 'team_id', 'position_name', 'is_ball_carrier',
            'x', 'y', 'vx', 'vy', etc.
            
    Returns:
        pd.DataFrame: DataFrame with new features added including:
            - is_teammate: Binary indicator if player is on same team as ball carrier
            - is_goalkeeper: Binary indicator if player is goalkeeper
            - distance_to_goal: Euclidean distance to goal
            - sin_angle_to_goal, cos_angle_to_goal: Trigonometric angles to goal
            - distance_to_ball: Euclidean distance to ball
            - sin_angle_to_ball, cos_angle_to_ball: Trigonometric angles to ball
            - cos_velocity_angle, sin_velocity_angle: Velocity angle relative to ball carrier
    """
    df = frame_df.copy()

    # Step 1: Identify main objects
    ball_row = df[df['id'] == 'ball']
    ball_carrier_row = df[df['is_ball_carrier'] == True]

    # Handle cases without ball or ball carrier
    if ball_row.empty or ball_carrier_row.empty:
        feature_cols_to_add = [
            'is_teammate', 'is_goalkeeper', 'distance_to_goal', 'sin_angle_to_goal',
            'cos_angle_to_goal', 'distance_to_ball', 'sin_angle_to_ball',
            'cos_angle_to_ball', 'cos_velocity_angle', 'sin_velocity_angle'
        ]
        for col in feature_cols_to_add:
            df[col] = 0.0
        return df

    ball_carrier = ball_carrier_row.iloc[0]
    ball = ball_row.iloc[0]
    
    # Goal position (left to right attack direction)
    goal_pos = np.array([52.5, 0.0])

    # Step 2: Calculate binary attributes
    df['is_teammate'] = (df['team_id'] == ball_carrier['team_id']).astype(float)
    df['is_goalkeeper'] = (df['position_name'] == 'GK').astype(float)

    # Step 3: Prepare data for vector calculations
    player_positions = df[['x', 'y']].values.astype(np.float64)
    player_velocities = df[['vx', 'vy']].values.astype(np.float64)
    ball_position = ball[['x', 'y']].values.astype(np.float64)
    carrier_velocity = ball_carrier[['vx', 'vy']].values.astype(np.float64)

    # Step 4: Features relative to goal
    vector_to_goal = goal_pos - player_positions
    df['distance_to_goal'] = np.linalg.norm(vector_to_goal, axis=1)
    angle_to_goal_rad = np.arctan2(vector_to_goal[:, 1], vector_to_goal[:, 0])
    df['sin_angle_to_goal'] = np.sin(angle_to_goal_rad)
    df['cos_angle_to_goal'] = np.cos(angle_to_goal_rad)

    # Step 5: Features relative to ball
    vector_to_ball = ball_position - player_positions
    df['distance_to_ball'] = np.linalg.norm(vector_to_ball, axis=1)
    angle_to_ball_rad = np.arctan2(vector_to_ball[:, 1], vector_to_ball[:, 0])
    df['sin_angle_to_ball'] = np.sin(angle_to_ball_rad)
    df['cos_angle_to_ball'] = np.cos(angle_to_ball_rad)

    # Step 6: Velocity angle features
    dot_product = np.sum(player_velocities * carrier_velocity, axis=1)
    norm_player = np.linalg.norm(player_velocities, axis=1)
    norm_carrier = np.linalg.norm(carrier_velocity)
    
    # Prevent division by zero
    denominator = (norm_player * norm_carrier) + 1e-8
    
    df['cos_velocity_angle'] = np.clip(dot_product / denominator, -1.0, 1.0)
    
    # Calculate sine using 2D vector cross product
    cross_product = (player_velocities[:, 0] * carrier_velocity[1] - 
                    player_velocities[:, 1] * carrier_velocity[0])
    df['sin_velocity_angle'] = np.clip(cross_product / denominator, -1.0, 1.0)

    return df

In [12]:
def _infer_feature_cols():
    """Infer the default feature columns for model input.
    
    Returns:
        list: List of feature column names to use for model training.
            Includes kinematic features (position, velocity, acceleration),
            binary features (teammate, goalkeeper), distance and angle features,
            and event type features.
    """
    return [
        'x', 'y', 'vx', 'vy', 'v', 'ax', 'ay', 'a',  # Kinematic features
        'is_teammate', 'is_goalkeeper',  # Binary features
        'distance_to_goal', 'sin_angle_to_goal', 'cos_angle_to_goal',  # Goal features
        'distance_to_ball', 'sin_angle_to_ball', 'cos_angle_to_ball',  # Ball features
        'cos_velocity_angle', 'sin_velocity_angle',  # Velocity angle features
        'type_id'  # Event type feature
    ]


In [13]:
def add_distance_ball_to_goal(df, goal_x=52.5, goal_y=0.0):
    """
    추적 데이터프레임(X_slice)의 각 프레임에 대해 
    공과 지정된 골대까지의 거리를 계산하여 새로운 컬럼으로 추가합니다.

    Args:
        df (pd.DataFrame): 'frame_id', 'id', 'x', 'y' 컬럼을 포함하는 X_slice.
        goal_x (float): 목표 골대의 x 좌표.
        goal_y (float): 목표 골대의 y 좌표.

    Returns:
        pd.DataFrame: 'distance_ball_to_goal' 컬럼이 추가된 DataFrame.
    """
    # 1. 각 프레임별 공의 위치를 찾습니다.
    ball_positions = df[df['id'] == 'ball'][['frame_id', 'x', 'y']]
    ball_positions = ball_positions.rename(columns={'x': 'ball_x', 'y': 'ball_y'})

    # 2. 원본 데이터프레임에 공의 위치 정보를 합칩니다.
    #    이제 모든 행은 자신이 속한 프레임의 공 위치를 알게 됩니다.
    df_merged = pd.merge(df, ball_positions, on='frame_id', how='left')

    # 3. 벡터 연산을 통해 거리를 한 번에 계산합니다.
    dx = goal_x - df_merged['ball_x']
    dy = goal_y - df_merged['ball_y']
    df_merged['distance_ball_to_goal'] = np.sqrt(dx**2 + dy**2)
    
    # 공 위치 정보를 위해 추가했던 컬럼은 제거합니다.
    df_merged = df_merged.drop(columns=['ball_x', 'ball_y'])

    return df_merged



In [14]:
event_df

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,...,position,team,jID,pID,tID,xID,player_code,time_diff,att_goal_count,def_goal_count
0,126285.0,0.0,0,1.0,0 days 00:00:01.520000,0 days 00:00:02.320000,4648.0,250102,77414.0,-1.0,...,CF,Home,20,250102,4648,10,H10,0.60,0,0
1,126285.0,2.0,2,1.0,0 days 00:00:03.120000,0 days 00:00:04.120000,4648.0,77414,250101.0,-1.0,...,CF,Home,10,77414,4648,9,H09,2.20,0,0
2,126285.0,4.0,4,1.0,0 days 00:00:05.320000,0 days 00:00:06.560000,4648.0,250101,62365.0,-1.0,...,CM,Home,14,250101,4648,6,H06,1.24,0,0
3,126285.0,NaN,6,1.0,0 days 00:00:06.560000,NaT,4648.0,62365,NaN,-1.0,...,CB,Home,6,62365,4648,2,H02,0.88,0,0
4,126285.0,6.0,7,1.0,0 days 00:00:10.680000,0 days 00:00:12.240000,4648.0,62365,500133.0,-1.0,...,CB,Home,6,62365,4648,2,H02,0.60,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,126285.0,3906.0,2880,2.0,0 days 00:52:45.840000,NaT,4648.0,500133,-1.0,-1.0,...,CB,Home,4,500133,4648,1,H01,2.16,2,0
1803,126285.0,NaN,2881,2.0,0 days 00:52:45.840000,NaT,4648.0,500133,NaN,-1.0,...,CB,Home,4,500133,4648,1,H01,2.16,2,0
1804,126285.0,3907.0,2882,2.0,0 days 00:52:47.040000,0 days 00:52:49.680000,4648.0,500138,500135.0,-1.0,...,CB,Home,15,500138,4648,13,H13,3.36,2,0
1805,126285.0,NaN,2884,2.0,0 days 00:52:49.720000,NaT,4648.0,500135,NaN,-1.0,...,RM,Home,11,500135,4648,5,H05,6.04,2,0


In [15]:
all_features_seqs = []
all_pressintensity_seqs = []
all_labels = []
all_pressed_ids = []
all_presser_ids = []
all_agent_orders = []
all_match_infos = []
FEATURE_FUNCTIONS = [
    add_distance_ball_to_goal
]
total_dict = {TEST_MATCH_ID: {}}
total_dict[TEST_MATCH_ID].update({
    'tracking_df': total_df,
    'event_df': event_df,
    'meta_data': meta_data,
    'Home': match_dict['teams']['Home'],
    'Away': match_dict['teams']['Away']
})
num_frames_to_sample = 10 # Number of frames to sample for each pressing situation
# Construct pressed_df if pressing intensity is greater than 0.9 for ball carrier.
# ball_carrier_df: schema
# "row": home team player_id, "column": away team player_id
# "probability_to_intercept"(len(row), len(column)): matrix of shape (home players, away players) representing pressing intensity each player exerts on opponent
pressed_dict = {}
ball_carrier_df = total_df[total_df['is_ball_carrier'] == True].copy() # Extract only frames with ball carrier: for detecting pressing situations (intensity, speed)
for idx, row in tqdm(ball_carrier_df.iterrows(), desc= "Get Pressing Intensity", miniters=len(ball_carrier_df)//10):                    
    if len(np.where(row['rows'] == row['id'])[0]) != 0: # If ball carrier is in home team
        pressed_axis = 'rows'
        presser_axis = 'columns'
        id_loc = np.where(row[pressed_axis] == row['id'])[0]  # Index of ball carrier in the row
        # Since it can be a nested list, apply tolist() twice
        pressing_values = row['probability_to_intercept'][id_loc].tolist()[0].tolist() # If ball carrier is in home team, pressing intensity for away players
    elif len(np.where(row['columns'] == row['id'])[0]) != 0: # If ball carrier is in away team
        pressed_axis = 'columns'
        presser_axis = 'rows'
        id_loc = np.where(row[pressed_axis] == row['id'])[0] # Index of ball carrier in the column
        pressing_values = [x[id_loc] for x in row['probability_to_intercept']] # If ball carrier is in away team, pressing intensity for home players
    else:
        continue
    if max(pressing_values) > 0.9:
        pressed_dict[idx] = {}
        pressed_dict[idx]['pressing_value'] = max(pressing_values)
        max_idx = pressing_values.index(max(pressing_values))
        pressed_dict[idx]['pressing_player'] = row[presser_axis][max_idx]
pressed_df = ball_carrier_df.loc[list(pressed_dict.keys())].copy()
pressed_df['pressing_values'] = [d['pressing_value'] for d in pressed_dict.values()]
pressed_df['pressing_player'] = [d.get('pressing_player') for d in pressed_dict.values()]

# Split pressing sequence
period_list = []
first_frames_list = []
for period_id in pressed_df['period_id'].unique():
    period_df = pressed_df[pressed_df['period_id']==period_id].copy()
    # If frame_id difference is greater than 125 frames, consider as a new sequence (i.e., not consecutive pressed rows)
    period_df['frame_diff'] = period_df['frame_id'].diff()
    period_df['sequence_id'] = (period_df['frame_diff'] > 125).cumsum()

    # For each sequence, get the first frame info to set X and Y
    first_frames = period_df.groupby('sequence_id', as_index=False)[['timestamp', 'period_id', 'frame_id', 'id', 'team_id', 'pressing_player']].first()

    # Extract only required columns from total_df
    lookup = total_df[['period_id', 'frame_id', 'id', 'v']]

    # pressing_player column matches total_df.id,
    # so align column names and merge
    first_frames = first_frames.merge(
        lookup.rename(columns={'id': 'pressing_player'}),
        on=['period_id', 'frame_id', 'pressing_player'],
        how='left'
    )

    # Only consider as pressing if the pressing player's speed is at least 2.0 m/s
    first_frames = first_frames[first_frames['v'] >= 2.0]
    period_list.append(first_frames)

first_frames_df = pd.concat(period_list, axis=0, ignore_index=True)    
first_frames_df['ball_ownership_changed'] = first_frames_df.apply(_check_pressing_success, axis=1, event_df=event_df, teams_dict=teams_dict) # window=150 is an example    
first_frames_list.append(first_frames_df)

# Create dictionary: extract and store by period_id (lookup table: optimize search space)
events_by_period = {period: df for period, df in event_df.groupby('period_id')}
tracking_by_period = {period: df for period, df in total_df.groupby('period_id')}

print(f"Match ID: {TEST_MATCH_ID} | Total Frames: {len(total_df)} | First Frames: {len(first_frames_df)}\n")
slices_to_process = []
sample_metadata = []
# Construct samples
for _, row in tqdm(first_frames_df.iterrows(), desc= "Get Samples"):#, miniters=len(first_frames)//10):
    try:
        period_id = row['period_id']
        frame_id = row['frame_id']
        timestamp = row['timestamp']
        label = int(row['ball_ownership_changed'])
        pressed_player = row['id']
        pressing_player = row['pressing_player']

        # Events that occurred in the 5 seconds before the pressing started
        event_period_df = events_by_period.get(period_id)
        window_event_df = event_period_df[
            (event_period_df['time_seconds'] >= timestamp - pd.Timedelta(seconds=5)) &
            (event_period_df['time_seconds'] <= timestamp)
        ]
        
        # Tracking data within 5 seconds before pressing started
        trace_period_df = tracking_by_period.get(period_id)
        window_trace_df = trace_period_df[
            (trace_period_df['timestamp'] >= timestamp - pd.Timedelta(seconds=5)) &
            (trace_period_df['timestamp'] <= timestamp)
        ].copy()

        counts_per_timestamp = window_trace_df.groupby('timestamp').size()
        pressing_frame_count = counts_per_timestamp.get(timestamp)
        valid_counts = counts_per_timestamp[counts_per_timestamp == pressing_frame_count]
        available_timestamps = valid_counts.index.tolist()

        # Always include the last frame (pressing moment), so temporarily remove it from the sampling pool: will add later
        if timestamp in available_timestamps:
            available_timestamps.remove(timestamp)

        # Sample (num_frames_to_sample-1) from (all-1) available timestamps
        if len(available_timestamps) >= (num_frames_to_sample - 1):
            # sampled_timestamps = random.sample(available_timestamps, num_frames_to_sample - 1)
            stride = len(available_timestamps) / (num_frames_to_sample - 1)
            sampled_timestamps = [available_timestamps[int(i * stride)] for i in range(num_frames_to_sample - 1)]
        else:
            # If less than num_frames_to_sample, use all available
            print(f"Warning : {TEST_MATCH_ID}-{period_id}-{frame_id} doesn't have {num_frames_to_sample} windows({len(available_timestamps)}).")
            sampled_timestamps = available_timestamps

        final_timestamps = sampled_timestamps + [timestamp]
        final_timestamps.sort()
        # X_slice = window_trace_df[window_trace_df['timestamp'].isin(final_timestamps)].copy()
        X_slice = total_df[total_df['timestamp'] == timestamp].copy()# single 프레임
        cols_to_flip = ['x', 'y', 'vx', 'vy', 'ax', 'ay']
        # Always press left -> right
        # If pressed player's team is home, flip: if carrier is in home team, mirror left-right (pressing team always attacks left to right)
        if X_slice.loc[(X_slice['frame_id']==frame_id) & (X_slice['is_ball_carrier']==True)]['team_id'].iloc[0] == match_dict['teams']['Home']['tID'].iloc[0]:
            for col in cols_to_flip:
                X_slice.loc[:, col] = -X_slice.loc[:, col]
            
        # Get Features
        # Generate kinematic features and event features per frame/agent
        window_event_df = window_event_df.copy()
        window_event_df["type_id"] += 1
        X_slice = pd.merge_asof(X_slice, 
                                window_event_df[["time_seconds", "type_id", "time_diff", "att_goal_count", "def_goal_count"]], 
                                left_on="timestamp", 
                                right_on="time_seconds", 
                                direction="forward")
        X_slice["type_id"] = X_slice["type_id"].ffill().fillna(0).astype(int)
        X_slice = X_slice.set_index('frame_id').groupby('frame_id', group_keys=False).apply(_generate_features)
        X_slice.reset_index(inplace=True)
    #     for feature_func in FEATURE_FUNCTIONS:
    #         X_slice = feature_func(X_slice)
 
        # Fill players if there are less than 22 players in the frame
        agents_rows = X_slice[(X_slice['frame_id']==frame_id) & (X_slice['is_ball_carrier']==True)]['rows'].values[0].tolist() # Home team
        agents_cols = X_slice[(X_slice['frame_id']==frame_id) & (X_slice['is_ball_carrier']==True)]['columns'].values[0].tolist() # Away team
        missing_cnt = 0
        num_missing_rows = NUM_TEAM_PLAYERS - len(agents_rows)
        if num_missing_rows > 0:
            for i in range(num_missing_rows):
                agents_rows.append(f"Missing_{missing_cnt}")
                missing_cnt += 1

        num_missing_cols = NUM_TEAM_PLAYERS - len(agents_cols)
        if num_missing_cols > 0:
            for i in range(num_missing_cols):
                agents_cols.append(f"Missing_{missing_cnt}")
                missing_cnt += 1
        agents_order = agents_rows + agents_cols
        
        # Ensure the player IDs are consistent and match num_agents (22 players + 1 ball)
        all_known_agents = set(X_slice['id'].unique())
        missing_agent_ids = [agent for agent in agents_order if agent not in all_known_agents and 'Missing' in agent]

        frame_lst = X_slice['frame_id'].unique()
        if missing_agent_ids:
            # Add missing player rows with zero values for each frame in X_slice
            missing_rows = []
            for missing_agent_id in missing_agent_ids:
                for frame in frame_lst:
                    missing_row = {col: 0 for col in X_slice.columns}  # Fill all columns with 0
                    missing_row['id'] = missing_agent_id  # Set the 'id' to the missing player's id
                    missing_row['frame_id'] = frame  # Set the frame_id for the current frame in the sequence
                    missing_rows.append(missing_row)
            if missing_rows:
                # Create a DataFrame for the missing rows and append to the slice
                missing_df = pd.DataFrame(missing_rows)
                X_slice = pd.concat([X_slice, missing_df], ignore_index=True)
        
        agents_order.append('ball')

        X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
        # X_slice['id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)

        # Sort the players by their ID to maintain a consistent order
        X_slice = X_slice.sort_values(by=['frame_id', 'id'])
        feature_cols = _infer_feature_cols()
        # Get the features
        x_tensor = torch.tensor(X_slice[feature_cols].values, dtype=torch.float32)

        X_slice_pressing = X_slice[X_slice['is_ball_carrier']==True]['probability_to_intercept']
        X_slice_pressing = X_slice_pressing.dropna()
        pressing_intensity_tensor = torch.tensor(np.stack(X_slice_pressing.map(lambda x: np.stack(x)).values), dtype=torch.float32)
        _, h, w = pressing_intensity_tensor.shape
        pad_h = NUM_TEAM_PLAYERS - h
        pad_w = NUM_TEAM_PLAYERS - w
        pressing_intensity_tensor = F.pad(pressing_intensity_tensor, (0, pad_w, 0, pad_h), "constant", 0)

        x_tensor = x_tensor.reshape(-1, NUM_AGENTS, len(feature_cols))
        y_tensor = torch.tensor(label, dtype=torch.long)

        # # Debug 
        # if x_tensor.isnan().any():
        #     print("Nan found", TEST_MATCH_ID, period_id, frame_id)
        #     continue
        # match_info = f"{TEST_MATCH_ID}-{period_id}-{frame_id}"
        

        # all_features_seqs.append(x_tensor)
        # all_pressintensity_seqs.append(pressing_intensity_tensor)
        # all_labels.append(y_tensor)
        # all_pressed_ids.append(pressed_player)
        # all_presser_ids.append(pressing_player)    
        # all_agent_orders.append(agents_order)
        # all_match_infos.append(match_info)
    
    except (FileNotFoundError, ValueError, KeyError) as e:
        error_type = type(e).__name__
        period_id = row.get('period_id', 'unknown')
        frame_id = row.get('frame_id', 'unknown')
        print(f"{error_type} in match {TEST_MATCH_ID}, period {period_id}, frame {frame_id}: {e}")
        continue
    except Exception as e:
        period_id = row.get('period_id', 'unknown')
        frame_id = row.get('frame_id', 'unknown')
        print(f"Unexpected error in match {TEST_MATCH_ID}, period {period_id}, frame {frame_id}: {e}")
        continue

Get Pressing Intensity: 0it [00:00, ?it/s]

Match ID: 126285 | Total Frames: 1921053 | First Frames: 214



Get Samples: 0it [00:00, ?it/s]

/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array.

Unexpected error in match 126285, period 1.0, frame 8140: shape '[-1, 23, 19]' is invalid for input of size 456
Unexpected error in match 126285, period 1.0, frame 8441: shape '[-1, 23, 19]' is invalid for input of size 456


...

/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array.

Unexpected error in match 126285, period 2.0, frame 134401: shape '[-1, 23, 19]' is invalid for input of size 456
Unexpected error in match 126285, period 2.0, frame 134645: shape '[-1, 23, 19]' is invalid for input of size 456


/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array.

Unexpected error in match 126285, period 2.0, frame 137812: shape '[-1, 23, 19]' is invalid for input of size 456
Unexpected error in match 126285, period 2.0, frame 139763: shape '[-1, 23, 19]' is invalid for input of size 456
Unexpected error in match 126285, period 2.0, frame 139979: shape '[-1, 23, 19]' is invalid for input of size 456


/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
/tmp/ipykernel_2308954/2606263565.py:194: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array.

In [16]:
X_slice.columns

Index(['frame_id', 'game_id', 'period_id', 'timestamp', 'ball_state',
       'ori_ball_owning_team_id', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'v', 'ax',
       'ay', 'az', 'a', 'id', 'team_id', 'position_name',
       'ball_owning_team_id', 'is_ball_carrier', 'time_to_intercept',
       'probability_to_intercept', 'columns', 'rows', 'player_code',
       'time_seconds', 'type_id', 'time_diff', 'att_goal_count',
       'def_goal_count', 'is_teammate', 'is_goalkeeper', 'distance_to_goal',
       'sin_angle_to_goal', 'cos_angle_to_goal', 'distance_to_ball',
       'sin_angle_to_ball', 'cos_angle_to_ball', 'cos_velocity_angle',
       'sin_velocity_angle'],
      dtype='object')

In [17]:
# 싱글 샘플링
X_slice

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle
2,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-11.621095,-8.787913,0.0,7.133403,...,1.0,0.0,64.720493,0.135783,0.990739,16.886859,-0.990721,-0.135910,0.545193,-0.838311
5,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.243807,9.821636,0.0,3.658966,...,1.0,0.0,76.377935,-0.128593,0.991698,36.549966,-0.966888,0.255202,0.568476,-0.822700
7,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-42.271835,-1.682267,0.0,0.871334,...,1.0,1.0,94.786764,0.017748,0.999842,37.043064,-0.643462,0.765478,0.718803,-0.695214
8,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.876117,-15.074832,0.0,4.056888,...,1.0,0.0,77.849611,0.193640,0.981073,14.431271,-0.723654,0.690163,0.829191,-0.558965
13,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-10.508006,0.404300,0.0,3.149458,...,1.0,0.0,63.009304,-0.006417,0.999979,26.145470,-0.991467,-0.130355,0.514781,-0.857322
15,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-35.561191,-1.728743,0.0,2.089413,...,1.0,0.0,88.078158,0.019627,0.999807,32.162691,-0.739656,0.672985,0.888197,-0.459463
16,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-7.714194,-25.744595,0.0,1.693566,...,1.0,0.0,65.486894,0.393126,0.919485,6.206132,0.036498,-0.999334,1.000000,0.000000
17,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-33.035939,-11.717439,0.0,3.284554,...,1.0,0.0,86.334786,0.135721,0.990747,23.580129,-0.585266,0.810842,0.726742,-0.686911
18,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-32.826194,0.422628,0.0,3.326097,...,1.0,0.0,85.327241,-0.004953,0.999988,32.101536,-0.808083,0.589068,0.563420,-0.826171
19,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-24.015056,-2.508657,0.0,4.643271,...,1.0,0.0,76.556170,0.032769,0.999463,25.128086,-0.915686,0.401896,0.467815,-0.883826


In [18]:
# 10 프레임 샘플링
X_slice

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle
2,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-11.621095,-8.787913,0.0,7.133403,...,1.0,0.0,64.720493,0.135783,0.990739,16.886859,-0.990721,-0.135910,0.545193,-0.838311
5,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.243807,9.821636,0.0,3.658966,...,1.0,0.0,76.377935,-0.128593,0.991698,36.549966,-0.966888,0.255202,0.568476,-0.822700
7,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-42.271835,-1.682267,0.0,0.871334,...,1.0,1.0,94.786764,0.017748,0.999842,37.043064,-0.643462,0.765478,0.718803,-0.695214
8,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.876117,-15.074832,0.0,4.056888,...,1.0,0.0,77.849611,0.193640,0.981073,14.431271,-0.723654,0.690163,0.829191,-0.558965
13,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-10.508006,0.404300,0.0,3.149458,...,1.0,0.0,63.009304,-0.006417,0.999979,26.145470,-0.991467,-0.130355,0.514781,-0.857322
15,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-35.561191,-1.728743,0.0,2.089413,...,1.0,0.0,88.078158,0.019627,0.999807,32.162691,-0.739656,0.672985,0.888197,-0.459463
16,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-7.714194,-25.744595,0.0,1.693566,...,1.0,0.0,65.486894,0.393126,0.919485,6.206132,0.036498,-0.999334,1.000000,0.000000
17,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-33.035939,-11.717439,0.0,3.284554,...,1.0,0.0,86.334786,0.135721,0.990747,23.580129,-0.585266,0.810842,0.726742,-0.686911
18,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-32.826194,0.422628,0.0,3.326097,...,1.0,0.0,85.327241,-0.004953,0.999988,32.101536,-0.808083,0.589068,0.563420,-0.826171
19,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-24.015056,-2.508657,0.0,4.643271,...,1.0,0.0,76.556170,0.032769,0.999463,25.128086,-0.915686,0.401896,0.467815,-0.883826


In [19]:
X_slice

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle
2,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-11.621095,-8.787913,0.0,7.133403,...,1.0,0.0,64.720493,0.135783,0.990739,16.886859,-0.990721,-0.135910,0.545193,-0.838311
5,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.243807,9.821636,0.0,3.658966,...,1.0,0.0,76.377935,-0.128593,0.991698,36.549966,-0.966888,0.255202,0.568476,-0.822700
7,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-42.271835,-1.682267,0.0,0.871334,...,1.0,1.0,94.786764,0.017748,0.999842,37.043064,-0.643462,0.765478,0.718803,-0.695214
8,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.876117,-15.074832,0.0,4.056888,...,1.0,0.0,77.849611,0.193640,0.981073,14.431271,-0.723654,0.690163,0.829191,-0.558965
13,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-10.508006,0.404300,0.0,3.149458,...,1.0,0.0,63.009304,-0.006417,0.999979,26.145470,-0.991467,-0.130355,0.514781,-0.857322
15,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-35.561191,-1.728743,0.0,2.089413,...,1.0,0.0,88.078158,0.019627,0.999807,32.162691,-0.739656,0.672985,0.888197,-0.459463
16,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-7.714194,-25.744595,0.0,1.693566,...,1.0,0.0,65.486894,0.393126,0.919485,6.206132,0.036498,-0.999334,1.000000,0.000000
17,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-33.035939,-11.717439,0.0,3.284554,...,1.0,0.0,86.334786,0.135721,0.990747,23.580129,-0.585266,0.810842,0.726742,-0.686911
18,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-32.826194,0.422628,0.0,3.326097,...,1.0,0.0,85.327241,-0.004953,0.999988,32.101536,-0.808083,0.589068,0.563420,-0.826171
19,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-24.015056,-2.508657,0.0,4.643271,...,1.0,0.0,76.556170,0.032769,0.999463,25.128086,-0.915686,0.401896,0.467815,-0.883826


In [20]:
window_event_df

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,...,position,team,jID,pID,tID,xID,player_code,time_diff,att_goal_count,def_goal_count
1800,126285.0,NaN,2878,2.0,0 days 00:52:45.120000,NaT,4648.0,250079,NaN,-1.0,...,GK,Home,1,250079,4648,0,H00,1.44,2,0
1801,126285.0,3905.0,2879,2.0,0 days 00:52:45.120000,NaT,4648.0,250079,-1.0,-1.0,...,GK,Home,1,250079,4648,0,H00,1.44,2,0
1802,126285.0,3906.0,2880,2.0,0 days 00:52:45.840000,NaT,4648.0,500133,-1.0,-1.0,...,CB,Home,4,500133,4648,1,H01,2.16,2,0
1803,126285.0,NaN,2881,2.0,0 days 00:52:45.840000,NaT,4648.0,500133,NaN,-1.0,...,CB,Home,4,500133,4648,1,H01,2.16,2,0
1804,126285.0,3907.0,2882,2.0,0 days 00:52:47.040000,0 days 00:52:49.680000,4648.0,500138,500135.0,-1.0,...,CB,Home,15,500138,4648,13,H13,3.36,2,0


In [21]:

def _get_frame_interaction_info(df):
    """
    (통합 헬퍼) 각 프레임에서 아래 두 정보를 모두 찾아 관련 데이터를 반환합니다.
    1. 볼 캐리어(actor)와 가장 가까운 수비수
    2. 공(ball)과 가장 가까운 수비수
    """
    results_by_frame = {}

    for frame_id, frame_df in df.groupby('frame_id'):
        ball_carrier = frame_df[frame_df['is_ball_carrier'] == True]
        ball = frame_df[frame_df['id'] == 'ball']

        if ball_carrier.empty or ball.empty:
            # 기본 정보가 없으면 모든 값을 NaN으로 설정
            # (필요한 모든 키를 NaN으로 초기화하는 것이 안전합니다)
            results_by_frame[frame_id] = { 'dist_def_to_actor': np.nan, 'dist_def_to_ball': np.nan}
            continue

        actor = ball_carrier.iloc[0]
        ball_row = ball.iloc[0]
        
        actor_pos = np.array([actor['x'], actor['y']])
        ball_pos = np.array([ball_row['x'], ball_row['y']])
        
        defenders = frame_df[(frame_df['team_id'] != actor['team_id']) & (frame_df['id'] != 'ball')]

        if defenders.empty:
            # 수비수가 없으면 수비수 관련 정보만 NaN으로 설정
            results_by_frame[frame_id] = { 'dist_def_to_actor': np.nan, 'dist_def_to_ball': np.nan}
            continue

        defender_positions = defenders[['x', 'y']].values

        # --- 계산 블록 1: 볼 캐리어(actor) 기준 ---
        dists_to_actor = np.linalg.norm(defender_positions - actor_pos, axis=1)
        idx_def_near_actor = np.argmin(dists_to_actor)
        def_near_actor = defenders.iloc[idx_def_near_actor]

        # --- 계산 블록 2: 공(ball) 기준 ---
        dists_to_ball = np.linalg.norm(defender_positions - ball_pos, axis=1)
        idx_def_near_ball = np.argmin(dists_to_ball)
        def_near_ball = defenders.iloc[idx_def_near_ball]
        distances_to_ball = np.linalg.norm(defenders[['x', 'y']].values - ball_pos, axis=1)

        # --- 모든 정보 취합 ---
        results_by_frame[frame_id] = {
            # 볼 캐리어 정보
            'carrier_speed': np.sqrt(actor['vx']**2 + actor['vy']**2),
            'carrier_x': actor['x'],
            'carrier_y': actor['y'],
            # 공 정보
            'ball_x': ball_row['x'],
            'ball_y': ball_row['y'],
            # '볼 캐리어'와 가장 가까운 수비수 정보
            'dist_def_to_actor': dists_to_actor[idx_def_near_actor],
            'speed_def_near_actor': np.sqrt(def_near_actor['vx']**2 + def_near_actor['vy']**2),
            'x_def_near_actor': def_near_actor['x'],
            'y_def_near_actor': def_near_actor['y'],
            # '공'과 가장 가까운 수비수 정보
            'dist_def_to_ball': dists_to_ball[idx_def_near_ball],
            'speed_def_near_ball': np.sqrt(def_near_ball['vx']**2 + def_near_ball['vy']**2),
            'x_def_near_ball': def_near_ball['x'],
            'y_def_near_ball': def_near_ball['y'],
            'nb_of_3m_radius' : np.sum(distances_to_ball <= 3.0),
            'nb_of_5m_radius': np.sum(distances_to_ball <= 5.0),
            'nb_of_10m_radius': np.sum(distances_to_ball <= 10.0)
        }

        features_df = pd.DataFrame.from_dict(results_by_frame, orient='index')
        features_df.index.name = 'frame_id'
        features_df.reset_index(inplace=True)

        # 2. X_slice에 merge
        merged_df = pd.merge(df, features_df, on='frame_id', how='left')
    return merged_df

In [22]:
_get_frame_interaction_info(X_slice)

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,speed_def_near_actor,x_def_near_actor,y_def_near_actor,dist_def_to_ball,speed_def_near_ball,x_def_near_ball,y_def_near_ball,nb_of_3m_radius,nb_of_5m_radius,nb_of_10m_radius
0,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-11.621095,-8.787913,0.0,7.133403,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
1,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.243807,9.821636,0.0,3.658966,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
2,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-42.271835,-1.682267,0.0,0.871334,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
3,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.876117,-15.074832,0.0,4.056888,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
4,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-10.508006,0.404300,0.0,3.149458,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
5,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-35.561191,-1.728743,0.0,2.089413,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
6,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-7.714194,-25.744595,0.0,1.693566,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
7,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-33.035939,-11.717439,0.0,3.284554,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
8,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-32.826194,0.422628,0.0,3.326097,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
9,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-24.015056,-2.508657,0.0,4.643271,...,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1


In [23]:
os.chdir('..')
import config as C

In [24]:
def add_distance_ball_to_center_of_goal(df):
    """
    추적 데이터프레임(X_slice)의 각 프레임에 대해 
    공과 지정된 골대까지의 거리를 계산하여 새로운 컬럼으로 추가합니다.

    Args:
        df (pd.DataFrame): 'frame_id', 'id', 'x', 'y' 컬럼을 포함하는 X_slice.
    Returns:
        pd.DataFrame: 'distance_ball_to_goal' 컬럼이 추가된 DataFrame.
    """
    # 1. 각 프레임별 공의 위치를 찾습니다.
    ball_positions = df[df['id'] == 'ball'][['frame_id', 'x', 'y']]
    ball_positions = ball_positions.rename(columns={'x': 'ball_x', 'y': 'ball_y'})

    # 2. 원본 데이터프레임에 공의 위치 정보를 합칩니다.
    #    이제 모든 행은 자신이 속한 프레임의 공 위치를 알게 됩니다.
    df_merged = pd.merge(df, ball_positions, on='frame_id', how='left')

    # 3. 벡터 연산을 통해 거리를 한 번에 계산합니다.
    dx = C.PITCH_X_MIN - df_merged['ball_x']
    dy = df_merged['ball_y']
    df_merged['distance_ball_to_goal'] = np.sqrt(dx**2 + dy**2)
    
    # 공 위치 정보를 위해 추가했던 컬럼은 제거합니다.
    df_merged = df_merged.drop(columns=['ball_x', 'ball_y'])

    return df_merged

In [25]:
def add_diff_ball_defender_sideline(df):
    df = df.copy()    
    ball_y_map = df.loc[df['id'] == 'ball'].set_index('frame_id')['y']

    df['ball_y'] = df['frame_id'].map(ball_y_map)    

    ball_to_sideline = np.minimum(
        (df['ball_y'] - C.PITCH_Y_MIN).abs(), 
        (C.PITCH_Y_MAX - df['ball_y']).abs()
    )
    defender_to_sideline = np.minimum(
        (df['y_def_near_ball'] - C.PITCH_Y_MIN).abs(), 
        (C.PITCH_Y_MAX - df['y_def_near_ball']).abs()
    )
    df['diff_ball_defender_sideline'] = (ball_to_sideline - defender_to_sideline).abs()
    df.drop(columns=['ball_y'], inplace=True)

    return df

In [26]:
add_diff_ball_defender_sideline(test)

NameError: name 'test' is not defined

In [ ]:
add_distance_ball_to_center_of_goal(X_slice)

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle,distance_ball_to_goal
0,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-15.256533,0.543596,0.0,-3.666936,...,0.0,67.758713,-0.008023,0.999968,20.688536,-0.271826,-0.962346,0.939633,-0.342183,18.063010
1,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-17.146921,16.611762,0.0,-5.532716,...,0.0,71.600588,-0.232006,0.972714,28.199751,-0.769221,-0.638983,0.898273,-0.439438,18.063010
2,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-43.652008,-0.938508,0.0,-0.584257,...,1.0,96.156588,0.009760,0.999952,9.442656,-0.438603,0.898681,0.480428,0.877034,18.063010
3,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-24.463659,-12.946000,0.0,-3.684163,...,0.0,78.044883,0.165879,0.986146,13.282104,0.592219,-0.805777,0.987180,-0.159609,18.063010
4,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-10.359025,0.009512,0.0,-2.616415,...,0.0,62.859026,-0.000151,1.000000,25.323774,-0.200981,-0.979595,0.813787,-0.581163,18.063010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,38.314007,-3.147688,0.0,0.685428,...,1.0,14.531013,0.216619,0.976256,56.819258,-0.393711,-0.919234,0.998135,-0.061046,46.258868
226,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-4.277243,-0.273143,0.0,4.231130,...,0.0,56.777900,0.004811,0.999988,27.022513,-0.934219,-0.356701,0.922442,-0.386136,46.258868
227,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-28.706064,12.727769,0.0,1.239774,...,0.0,82.197451,-0.154844,0.987939,41.005920,-0.932691,0.360677,-0.240256,-0.970710,46.258868
228,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-3.431331,-22.009390,0.0,-2.074155,...,0.0,60.105965,0.366176,0.930545,11.056365,-0.317346,-0.948310,0.662319,0.749222,46.258868


In [ ]:
def add_dist_diff_sideline_ball_defender(df):
    df = df.copy()
    
    ball_y_map = df.loc[df['id'] == 'ball'].set_index('frame_id')['y']

    df['ball_y'] = df['frame_id'].map(ball_y_map)
    

    ball_to_sideline = np.minimum(
        (df['ball_y'] - -34).abs(), 
        (34 - df['ball_y']).abs()
    )
    defender_to_sideline = np.minimum(
        (df['y_def_near_ball'] - -34).abs(), 
        (34 - df['y_def_near_ball']).abs()
    )
    df['dist_diff_sideline'] = (ball_to_sideline - defender_to_sideline).abs()
    df.drop(columns=['ball_y'], inplace=True)

    return df

add_dist_diff_sideline_ball_defender(test)

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,x_def_near_actor,y_def_near_actor,dist_def_to_ball,speed_def_near_ball,x_def_near_ball,y_def_near_ball,nb_of_3m_radius,nb_of_5m_radius,nb_of_10m_radius,dist_diff_sideline
0,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-15.256533,0.543596,0.0,-3.666936,...,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4,2.625584
1,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-17.146921,16.611762,0.0,-5.532716,...,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4,2.625584
2,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-43.652008,-0.938508,0.0,-0.584257,...,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4,2.625584
3,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-24.463659,-12.946000,0.0,-3.684163,...,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4,2.625584
4,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-10.359025,0.009512,0.0,-2.616415,...,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4,2.625584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,38.314007,-3.147688,0.0,0.685428,...,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1,8.934435
226,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-4.277243,-0.273143,0.0,4.231130,...,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1,8.934435
227,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-28.706064,12.727769,0.0,1.239774,...,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1,8.934435
228,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-3.431331,-22.009390,0.0,-2.074155,...,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1,8.934435


In [ ]:
_get_frame_interaction_info(X_slice).iloc[:,44:]

,carrier_y,ball_x,ball_y,dist_def_to_actor,speed_def_near_actor,x_def_near_actor,y_def_near_actor,dist_def_to_ball,speed_def_near_ball,x_def_near_ball,y_def_near_ball,nb_of_3m_radius,nb_of_5m_radius,nb_of_10m_radius
0,-2.625500,-35.166071,-5.080084,1.378418,4.584840,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4
1,-2.625500,-35.166071,-5.080084,1.378418,4.584840,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4
2,-2.625500,-35.166071,-5.080084,1.378418,4.584840,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4
3,-2.625500,-35.166071,-5.080084,1.378418,4.584840,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4
4,-2.625500,-35.166071,-5.080084,1.378418,4.584840,-34.041915,-2.45450,2.856119,4.584840,-34.041915,-2.454500,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,-25.744595,-13.916191,-25.518082,5.682840,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
226,-25.744595,-13.916191,-25.518082,5.682840,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
227,-25.744595,-13.916191,-25.518082,5.682840,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1
228,-25.744595,-13.916191,-25.518082,5.682840,6.968841,-3.431331,-22.00939,9.072479,5.915443,-15.492813,-16.583646,0,0,1


In [ ]:
    ball_positions = df[df['id'] == 'ball'][['frame_id', 'x', 'y']]
    ball_positions = ball_positions.rename(columns={'x': 'ball_x', 'y': 'ball_y'})
    

In [ ]:
X_slice[X_slice['frame_id']==frame_id]

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle
209,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-11.621095,-8.787913,0.0,7.133403,...,1.0,0.0,64.720493,0.135783,0.990739,16.886859,-0.990721,-0.135910,0.545193,-0.838311
212,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.243807,9.821636,0.0,3.658966,...,1.0,0.0,76.377935,-0.128593,0.991698,36.549966,-0.966888,0.255202,0.568476,-0.822700
214,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-42.271835,-1.682267,0.0,0.871334,...,1.0,1.0,94.786764,0.017748,0.999842,37.043064,-0.643462,0.765478,0.718803,-0.695214
215,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-23.876117,-15.074832,0.0,4.056888,...,1.0,0.0,77.849611,0.193640,0.981073,14.431271,-0.723654,0.690163,0.829191,-0.558965
220,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-10.508006,0.404300,0.0,3.149458,...,1.0,0.0,63.009304,-0.006417,0.999979,26.145470,-0.991467,-0.130355,0.514781,-0.857322
222,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-35.561191,-1.728743,0.0,2.089413,...,1.0,0.0,88.078158,0.019627,0.999807,32.162691,-0.739656,0.672985,0.888197,-0.459463
223,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-7.714194,-25.744595,0.0,1.693566,...,1.0,0.0,65.486894,0.393126,0.919485,6.206132,0.036498,-0.999334,1.000000,0.000000
224,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-33.035939,-11.717439,0.0,3.284554,...,1.0,0.0,86.334786,0.135721,0.990747,23.580129,-0.585266,0.810842,0.726742,-0.686911
225,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-32.826194,0.422628,0.0,3.326097,...,1.0,0.0,85.327241,-0.004953,0.999988,32.101536,-0.808083,0.589068,0.563420,-0.826171
226,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-24.015056,-2.508657,0.0,4.643271,...,1.0,0.0,76.556170,0.032769,0.999463,25.128086,-0.915686,0.401896,0.467815,-0.883826


In [ ]:
def _get_frame_interaction_info(df):
    """
    (통합 헬퍼) 각 프레임에서 아래 두 정보를 모두 찾아 관련 데이터를 반환합니다.
    1. 볼 캐리어(actor)와 가장 가까운 수비수
    2. 공(ball)과 가장 가까운 수비수
    """
    results_by_frame = {}

    for frame_id, frame_df in df.groupby('frame_id'):
        ball_carrier = frame_df[frame_df['is_ball_carrier'] == True]
        ball = frame_df[frame_df['id'] == 'ball']

        if ball_carrier.empty or ball.empty:
            # 기본 정보가 없으면 모든 값을 NaN으로 설정
            # (필요한 모든 키를 NaN으로 초기화하는 것이 안전합니다)
            results_by_frame[frame_id] = { 'dist_def_to_actor': np.nan, 'dist_def_to_ball': np.nan}
            continue

        actor = ball_carrier.iloc[0]
        ball_row = ball.iloc[0]
        
        actor_pos = np.array([actor['x'], actor['y']])
        ball_pos = np.array([ball_row['x'], ball_row['y']])
        
        defenders = frame_df[(frame_df['team_id'] != actor['team_id']) & (frame_df['id'] != 'ball')]

        if defenders.empty:
            # 수비수가 없으면 수비수 관련 정보만 NaN으로 설정
            results_by_frame[frame_id] = { 'dist_def_to_actor': np.nan, 'dist_def_to_ball': np.nan}
            continue

        defender_positions = defenders[['x', 'y']].values

        # --- 계산 블록 1: 볼 캐리어(actor) 기준 ---
        dists_to_actor = np.linalg.norm(defender_positions - actor_pos, axis=1)
        idx_def_near_actor = np.argmin(dists_to_actor)
        def_near_actor = defenders.iloc[idx_def_near_actor]

        # --- 계산 블록 2: 공(ball) 기준 ---
        dists_to_ball = np.linalg.norm(defender_positions - ball_pos, axis=1)
        idx_def_near_ball = np.argmin(dists_to_ball)
        def_near_ball = defenders.iloc[idx_def_near_ball]
        
        # --- 모든 정보 취합 ---
        results_by_frame[frame_id] = {
            # 볼 캐리어 정보
            'carrier_speed': np.sqrt(actor['vx']**2 + actor['vy']**2),
            'carrier_x': actor['x'],
            'carrier_y': actor['y'],
            # 공 정보
            'ball_x': ball_row['x'],
            'ball_y': ball_row['y'],
            # '볼 캐리어'와 가장 가까운 수비수 정보
            'dist_def_to_actor': dists_to_actor[idx_def_near_actor],
            'speed_def_near_actor': np.sqrt(def_near_actor['vx']**2 + def_near_actor['vy']**2),
            'x_def_near_actor': def_near_actor['x'],
            'y_def_near_actor': def_near_actor['y'],
            # '공'과 가장 가까운 수비수 정보
            'dist_def_to_ball': dists_to_ball[idx_def_near_ball],
            'speed_def_near_ball': np.sqrt(def_near_ball['vx']**2 + def_near_ball['vy']**2),
            'x_def_near_ball': def_near_ball['x'],
            'y_def_near_ball': def_near_ball['y'],
        }
    return results_by_frame

In [ ]:
# --- 1. 최단 거리 수비수와의 거리 ---
def add_closest_defender_distance(df):
    df = df.copy()
    # 공통 함수를 한 번만 호출
    results_by_frame = _get_frame_interaction_info(df)
    
    # map을 사용해 각 행의 frame_id에 맞는 결과를 매핑하고, 'distance' 값만 추출
    df['closest_defender_dist'] = df['frame_id'].map(results_by_frame).str.get('dist_def_to_actor')
    return df
# --- 2. 최단 거리 수비수의 속도 ---
def add_closest_defender_speed(df):
    df = df.copy()
    results_by_frame = _get_frame_interaction_info(df)
    
    # 'defender_speed' 값만 추출
    df['closest_defender_speed'] = df['frame_id'].map(results_by_frame).str.get('speed_def_near_actor')
    return df
# --- 3. 행위자와 최단거리 수비수의 속도 차이 ---
def add_speed_diff_actor_defender(df):
    df = df.copy()
    results_by_frame = _get_frame_interaction_info(df)
    
    # map으로 결과 딕셔너리 시리즈를 가져옴
    results_series = df['frame_id'].map(results_by_frame)
    
    # 각 속도 값을 추출
    carrier_speeds = results_series.str.get('carrier_speed')
    defender_speeds = results_series.str.get('speed_def_near_actor')
    
    # 속도 차이 계산
    df['speed_diff_actor_defender'] = (carrier_speeds - defender_speeds).abs()
    return df


# --- 4. 공과 가장 가까운 수비수와 사이드라인 사이의 거리 ---
def add_dist_def_near_ball_to_sideline(df):
    """'공'과 가장 가까운 수비수와, 그 수비수와 가장 가까운 사이드라인 사이의 거리를 계산합니다."""
    df = df.copy()
    results_by_frame = _get_frame_interaction_info(df)
    
    # '공'과 가까운 수비수의 y좌표('y_def_near_ball')를 가져옴
    defender_y = df['frame_id'].map(results_by_frame).str.get('y_def_near_ball')
    
    df['dist_def_near_ball_to_sideline'] = np.minimum(defender_y - -34, 34 - defender_y).abs()
    return df


# --- 5. 공과 가장 가까운 수비수와 골라인 사이의 거리 ---
def add_dist_def_near_ball_to_goaline(df):
    """'공'과 가장 가까운 수비수와 가장 가까운 수비수와 수비 골라인 사이의 거리를 계산합니다."""
    df = df.copy()
    results_by_frame = _get_frame_interaction_info(df)

    # '볼 캐리어'와 가까운 수비수의 x좌표('x_def_near_actor')를 가져옴
    defender_x = df['frame_id'].map(results_by_frame).str.get('x_def_near_ball')
    
    df['dist_def_near_ball_to_goaline'] = (defender_x - -52.5).abs()
    return df

# --- 6. (공에 가장 가까운 수비수와 골라인 사이의 거리, 공과 골라인 사이의 거리), 두 거리의 차이 ---
def add_diff_ball_defender_goalline(df):

    df = df.copy()
    results_by_frame = _get_frame_interaction_info(df)
    # '공에 가장 가까운 수비수와 골라인 사이의 거리를 가져옴
 
    defender_x = df['frame_id'].map(results_by_frame).str.get('x_def_near_ball')
    defender_to_goaline = (defender_x - -52.5).abs()

    # 공 정보
    ball_x = df['frame_id'].map(results_by_frame).str.get('ball_x')
    ball_to_goaline = (ball_x - -52.5).abs()
    df['diff_ball_defender_goalline'] = (defender_to_goaline - ball_to_goaline).abs()
    return df

# --- 7. (공에 가장 가까운 수비수와 사이드라인 사이의 거리, 공과 사이드라인 사이의 거리), 두 거리의 차이 ---
def add_diff_ball_defender_sideline(df):

    df = df.copy()
    results_by_frame = _get_frame_interaction_info(df)
    # '공에 가장 가까운 수비수와 사이드라인 사이의 거리를 가져옴
 
    defender_y = df['frame_id'].map(results_by_frame).str.get('y_def_near_ball')
    defender_to_sideline = np.minimum(defender_y - -34, 34 - defender_y).abs()

    # 공 정보
    ball_y = df['frame_id'].map(results_by_frame).str.get('ball_y')
    ball_to_sideline = np.minimum(ball_y - -34, 34 - ball_y).abs()

    df['diff_ball_defender_sideline'] = (defender_to_sideline - ball_to_sideline).abs()
    return df

print(add_closest_defender_distance(X_slice)[['closest_defender_dist']].iloc[0])
print(add_closest_defender_speed(X_slice)[['closest_defender_speed']].iloc[0])
print(add_speed_diff_actor_defender(X_slice)[['speed_diff_actor_defender']].iloc[0])
print(add_dist_def_near_ball_to_sideline(X_slice)[['dist_def_near_ball_to_sideline']].iloc[0])
print(add_dist_def_near_ball_to_goaline(X_slice)[['dist_def_near_ball_to_goaline']].iloc[0])
print(add_diff_ball_defender_goalline(X_slice)[['diff_ball_defender_goalline']].iloc[0])
print(add_diff_ball_defender_sideline(X_slice)[['diff_ball_defender_sideline']].iloc[0])



closest_defender_dist    1.378418
Name: 2, dtype: float64
closest_defender_speed    4.58484
Name: 2, dtype: float64
speed_diff_actor_defender    1.630903
Name: 2, dtype: float64
dist_def_near_ball_to_sideline    31.5455
Name: 2, dtype: float64
dist_def_near_ball_to_goaline    18.458085
Name: 2, dtype: float64
diff_ball_defender_goalline    1.124156
Name: 2, dtype: float64
diff_ball_defender_sideline    2.625584
Name: 2, dtype: float64


In [ ]:
event_df_dict = defaultdict(list)
for row in melted.to_dict(orient='records'):
    event_df_dict[row['event_id']].append(row)
event_ids = list(event_df_dict.keys())
event_ids

[146737]

In [ ]:
from matplotlib.colors import TwoSlopeNorm

# Generate Pitch Control for predictor models
def calculate_model_pitch_control(results_df, event_id):
    event_df = results_df[results_df["event_id"] == event_id].copy()
    print(event_df)
    # event_df["x"] = event_df["act_x"].copy()
    # event_df["y"] = event_df["act_y"].copy()


    # event_df.loc[event_df["team_on_ball"] == False, "x"] = config.field_length - event_df.loc[event_df["team_on_ball"] == False, "x"].values
    # event_df.loc[event_df["team_on_ball"] == False, "y"] = config.field_width - event_df.loc[event_df["team_on_ball"] == False, "y"].values
    print(event_df.iloc[0], event_df)
    PPCFa, xgrid, ygrid = pc.generate_pitch_control_for_event(
        event_df.iloc[0], event_df, params, 
        field_dimen=(105, 68,), 
        n_grid_cells_x=52, n_grid_cells_y=34, offsides=False, #52.2
    )
    
    fig, ax = pc.plot_pitchcontrol_for_event(PPCFa, event_df.iloc[0], event_df)
    fig.set_size_inches(12,8)

    return PPCFa, ax, event_df, xgrid, ygrid

def plot_pitchcontrol(event_id, PPCF, locs, alpha = 0.7, include_player_velocities=True, annotate=False, field_dimen = (105.0, 68)):
    #highlight = events[events["event_id"] == event_id].iloc[0]

    if False:
        cond = (locs["period_id"] == 1)
        locs.loc[cond, "x"] = (
            105 - locs.loc[cond, "x"].values
        )
        locs.loc[cond, "y"] = (
            68 - locs.loc[cond, "y"].values
        )
    
    row = {}
    for r in locs.itertuples():
        row[f"{r.key}_x"] = r.x
        row[f"{r.key}_y"] = r.y
    # row["B00_x"] = locs.iloc[0].ballx
    # row["B00_y"] = locs.iloc[0].bally
    trace = pd.DataFrame.from_dict(row, orient='index').T


    # 컬러바 생성
    # cbar = fig.colorbar(pc, orientation='vertical')
    # cbar.set_label('Probability')
    
    norm = TwoSlopeNorm(vmin=0.0, vcenter=0.5, vmax=np.max(PPCF))
    # pc = ax.imshow(np.flipud(PPCF), extent=(0, field_dimen[0], 0, field_dimen[1]),
    #             norm=norm, cmap='bwr', alpha=0.5)

    pc = ax.imshow(np.flipud(PPCF), extent=(0, field_dimen[0], 0, field_dimen[1]), 
                vmin=0.0, vmax=1.0, cmap='bwr', alpha=0.5)
    
    # for i in range(PPCF.shape[0]):
    #     for j in range(PPCF.shape[1]):
    #         val = PPCF[i, j]
    #         if val > 0.01:  # 너무 작은 값은 생략
    #             ax.text(
    #                 j * (field_dimen[0] / PPCF.shape[1]) + (field_dimen[0] / (2 * PPCF.shape[1])),  # x
    #                 i * (field_dimen[1] / PPCF.shape[0]) + (field_dimen[1] / (2 * PPCF.shape[0])),  # y
    #                 f"{val:.2f}",
    #                 color='black',
    #                 ha='center',
    #                 va='center',
    #                 fontsize=6,
    #                 alpha=0.8
    #             )
    norm = TwoSlopeNorm(vmin=0.0, vcenter=0.5, vmax=1.0)

    # pc = ax.imshow(np.flipud(PPCF), extent=(0, field_dimen[0], 0, field_dimen[1]),cmap='bwr',alpha=0.5, norm=norm)
    # cbar = fig.colorbar(pc, orientation='vertical')
    # cbar.set_label('Probability')

    return fig, ax

In [35]:

def flatten_df(merged_df):    
    # 1. ball 위치를 따로 추출
    ball_df = merged_df[merged_df['id'] == 'ball'][['frame_id', 'x', 'y']].rename(columns={'x': 'ballx', 'y': 'bally'})


    # 2. 공 소유 선수를 제외한 나머지 행들만 필터링 (또는 모든 행을 쓸 수 있음)
    player_df = merged_df[merged_df['id'] != 'ball'].copy()

    # 3. 각 선수의 위치 정보에 해당 프레임의 공 위치 병합
    melted = pd.merge(player_df, ball_df, on='frame_id', how='left')

    # 4. 필요한 컬럼만 추려내기 (원하시면 순서 조정 가능)
    melted = melted[[
        'player_code',
        'period_id',
        'frame_id',
        'team_id',
        'ball_owning_team_id',
        'is_ball_carrier',
        'position_name',
        'x', 'y', 'vx', 'vy',
        'ballx', 'bally'
    ]].rename(columns={
        'frame_id': 'event_id',
        'player_code': 'key',
        'is_ball_carrier': 'player_on_ball',
        'position_name': 'position'
    })
    melted['team_on_ball'] = melted['team_id'] == melted['ball_owning_team_id']
    melted['team_id'] = melted['team_id'].astype(int)
    melted['team_on_ball'] = melted['team_id'] == melted['ball_owning_team_id']

    melted["x"] = melted["x"] + C.PITCH_X_MAX
    melted["y"] = melted["y"] + C.PITCH_Y_MAX
    melted["ballx"] = melted["ballx"] + C.PITCH_X_MAX
    melted["bally"] = melted["bally"] + C.PITCH_Y_MAX
    return melted

In [34]:
traces_df[['team_id']]

,team_id
0,4643
1,4648
2,4648
3,4648
4,4648
...,...
225,4648
226,4648
227,4648
228,4643


In [38]:
traces_df = pd.read_csv('/home/express-v2/final_x_slice.csv')
melted_df = flatten_df(traces_df)
melted_df.head(22)

,key,period_id,event_id,team_id,ball_owning_team_id,player_on_ball,position,x,y,vx,vy,ballx,bally,team_on_ball
0,A07,1.0,45,4643,4643,False,RM,52.997143,48.2084,-0.486363,-0.154585,56.924036,32.1319,True
1,H05,1.0,45,4648,4643,False,LM,52.223786,56.8477,0.702361,-1.143280,56.924036,32.1319,False
2,H00,1.0,45,4648,4643,False,GK,3.708321,34.7462,0.646797,0.266104,56.924036,32.1319,False
3,H06,1.0,45,4648,4643,False,CM,43.670143,36.0174,1.823958,0.196262,56.924036,32.1319,False
4,H10,1.0,45,4648,4643,False,CF,52.805143,24.9023,0.966062,0.057131,56.924036,32.1319,False
5,A10,1.0,45,4643,4643,False,CF,51.906429,22.9487,-1.096368,0.242238,56.924036,32.1319,True
6,A02,1.0,45,4643,4643,False,CB,73.407964,46.9135,0.481658,0.530276,56.924036,32.1319,True
7,A06,1.0,45,4643,4643,False,CM,63.546321,36.6995,0.941680,-0.617706,56.924036,32.1319,True
8,A03,1.0,45,4643,4643,False,CB,77.159786,33.3056,0.392805,-0.049781,56.924036,32.1319,True
9,A00,1.0,45,4643,4643,False,GK,102.411857,33.4769,-0.508548,0.001429,56.924036,32.1319,True


In [39]:
import os

os.cpu_count()

80

In [ ]:
event_df = melted[melted["event_id"] == 146737].copy()
print(event_df)
# event_df["x"] = event_df["act_x"].copy()
# event_df["y"] = event_df["act_y"].copy()


# event_df.loc[event_df["team_on_ball"] == False, "x"] = config.field_length - event_df.loc[event_df["team_on_ball"] == False, "x"].values
# event_df.loc[event_df["team_on_ball"] == False, "y"] = config.field_width - event_df.loc[event_df["team_on_ball"] == False, "y"].values
print(event_df.iloc[0], event_df)
PPCFa, xgrid, ygrid = pc.generate_pitch_control_for_event(
    event_df.iloc[0], event_df, params, 
    field_dimen=(105, 68,), 
    n_grid_cells_x=52, n_grid_cells_y=34, offsides=False, #52.2
)

    key  period_id  event_id team_id  player_on_ball position          x  \
0   H18        2.0    146737    4648           False       CF  40.878905   
1   H14        2.0    146737    4648           False       LM  29.256193   
2   H00        2.0    146737    4648           False       GK  10.228165   
3   H06        2.0    146737    4648           False       CM  28.623883   
4   H19        2.0    146737    4648           False       CF  41.991994   
5   H01        2.0    146737    4648           False       CB  16.938809   
6   H05        2.0    146737    4648            True       RM  44.785806   
7   H13        2.0    146737    4648           False       CB  19.464061   
8   H04        2.0    146737    4648           False       LB  19.673806   
9   H17        2.0    146737    4648           False       CM  28.484944   
10  H03        2.0    146737    4648           False       RB  18.291529   
11  A01        2.0    146737    4648           False       LB  21.694536   
12  A06     

In [ ]:
event_df_dict = defaultdict(list)
for row in melted_df.to_dict(orient='records'):
    event_df_dict[row['event_id']].append(row)
event_ids = list(event_df_dict.keys())
event_ids
# event_df = pd.DataFrame(event_df_dict[146737])
# PPCFa, xgrid, ygrid = pc.generate_pitch_control_for_event(
#     event_df.iloc[0], event_df, params,
#     field_dimen=(105, 68),
#     n_grid_cells_x=52, n_grid_cells_y=34, offsides=False,
# )

[146612,
 146625,
 146639,
 146653,
 146667,
 146681,
 146695,
 146709,
 146723,
 146737]

In [30]:
melted_df

,key,period_id,event_id,team_id,ball_owning_team_id,player_on_ball,position,x,y,vx,vy,ballx,bally,team_on_ball
0,A07,1.0,45,4643,4643,False,RM,52.997143,48.2084,-0.486363,-0.154585,56.924036,32.1319,True
1,H05,1.0,45,4648,4643,False,LM,52.223786,56.8477,0.702361,-1.143280,56.924036,32.1319,False
2,H00,1.0,45,4648,4643,False,GK,3.708321,34.7462,0.646797,0.266104,56.924036,32.1319,False
3,H06,1.0,45,4648,4643,False,CM,43.670143,36.0174,1.823958,0.196262,56.924036,32.1319,False
4,H10,1.0,45,4648,4643,False,CF,52.805143,24.9023,0.966062,0.057131,56.924036,32.1319,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,A01,1.0,170,4643,4643,False,RB,54.650571,65.5725,0.578649,0.949063,68.429679,36.7350,True
216,H02,1.0,170,4648,4643,False,CB,39.166500,32.0666,0.047465,2.169817,68.429679,36.7350,False
217,H03,1.0,170,4648,4643,False,RB,40.314857,17.5103,0.642860,-0.234910,68.429679,36.7350,False
218,H09,1.0,170,4648,4643,False,CF,65.505643,41.4186,2.541060,-2.672537,68.429679,36.7350,False


In [32]:
PPCFa, ax, event_df, xgrid, ygrid = calculate_model_pitch_control(melted_df, event_id=146737)

NameError: name 'calculate_model_pitch_control' is not defined

In [ ]:
def sum_pitch_control_vectorized(PPCFa, event_row, xgrid, ygrid, radius_m=4.0):
    ball_x = event_row['ballx']
    ball_y = event_row['bally']

    # 1. xgrid, ygrid로 meshgrid 만들기 → 전체 셀의 좌표 (2D)
    X, Y = np.meshgrid(xgrid, ygrid)  # shape = (rows, cols)

    # 2. 각 셀 중심과 공 위치 간 거리 계산
    distance = np.sqrt((X - ball_x)**2 + (Y - ball_y)**2)

    # 3. radius 안에 있는 셀 마스크
    in_radius_mask = distance <= radius_m

    # 4. 유효한 셀 마스크 (NaN 제외)
    valid_mask = ~np.isnan(PPCFa)

    # 5. 두 조건 모두 만족하는 셀만 합산
    final_mask = in_radius_mask & valid_mask
    return PPCFa[final_mask].sum()

In [ ]:
sum_pitch_control(X_slice)

NameError: name 'sum_pitch_control' is not defined

In [ ]:
sum_4m_radius = sum_pitch_control_vectorized(
    PPCFa,
    event_df.iloc[0], # 공의 위치 정보를 담고 있는 이벤트 행
    xgrid,
    ygrid,
    radius_m = 4.0 
)
sum_4m_radius

3.298762424586351

In [ ]:
import pandas as pd
# pc (PitchControl) 라이브러리와 C (상수)는 임포트되어 있다고 가정합니다.
# import PitchControl as pc
# import config as C

def add_pitch_control(df, params, radius_m=4.0):
    """
    X_slice와 같은 추적 데이터프레임을 받아 각 프레임의 Pitch Control 합계를 계산하고,
    'sum_pitch_control' 컬럼을 추가합니다. 멀티프로세싱은 사용하지 않습니다.

    Args:
        df (pd.DataFrame): X_slice와 같은 형태의 추적 데이터프레임.
        params: Pitch Control 모델의 파라미터.
        radius_m (float): 공 주변 몇 미터 반경의 Pitch Control을 합산할지 결정.

    Returns:
        pd.DataFrame: 'sum_pitch_control' 컬럼이 추가된 DataFrame.
    """
    # 1. 입력된 df(X_slice)를 Pitch Control 계산에 필요한 형태로 변환합니다.
    melted_df = flatten_df(df)
    
    # 계산 결과를 프레임 ID별로 저장할 딕셔너리
    pc_results = {}

    # 2. 멀티프로세싱 대신, melted_df를 event_id(frame_id)로 그룹화하여 순차적으로 처리합니다.
    for event_id, event_df in melted_df.groupby('event_id'):
        try:
            # 3. single_event_pitch_control 함수의 핵심 로직을 그대로 실행합니다.
            PPCFa, xgrid, ygrid = pc.generate_pitch_control_for_event(
                event_df.iloc[0], event_df, params,
                field_dimen=(105,68),
                n_grid_cells_x=52, n_grid_cells_y=34, offsides=False,
            )
            ball_x = event_df.iloc[0]['ballx']
            ball_y = event_df.iloc[0]['bally']
            
            X, Y = np.meshgrid(xgrid, ygrid)
            distance = np.sqrt((X - ball_x)**2 + (Y - ball_y)**2)
            final_mask = (distance <= radius_m) & (~np.isnan(PPCFa))
            
            summed_pc = PPCFa[final_mask].sum() if np.any(final_mask) else 0.0
            pc_results[event_id] = summed_pc
            
        except Exception as e:
            print(f"[event_id {event_id}] pitch control 계산 실패: {e}")
            pc_results[event_id] = 0.0

    # 4. 계산된 결과를 원래 df(X_slice)의 'frame_id'에 맞게 매핑하여 새 컬럼으로 추가합니다.
    df['sum_pitch_control'] = df['frame_id'].map(pc_results)
    
    return df

In [ ]:
add_pitch_control(X_slice, params, radius_m=4.0)

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows,player_code,time_seconds,type_id,time_diff,att_goal_count,def_goal_count,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle,goal_diff,nb_of_3m_radius,sum_pitch_control
0,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-29.767707,2.009037,0.0,-4.386689,-1.398341,0.0,4.604172,-2.842841,4.107391,0.0,4.995238,139210,316,LB,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A01,0 days 00:52:45.120000,9,1.44,2.0,0.0,1.0,0.0,82.292234,-0.024413,0.999702,8.910554,-0.795587,-0.605839,0.920331,-0.391141,2.0,2.0,3.298762
1,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-11.954909,-8.306194,0.0,-0.114114,0.156071,0.0,0.193340,0.597585,1.555941,0.0,1.666752,143410,316,CM,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A06,0 days 00:52:45.120000,9,1.44,2.0,0.0,1.0,0.0,64.987908,0.127811,0.991798,23.434288,0.137666,-0.990479,0.662853,0.748749,2.0,2.0,3.298762
2,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-15.256533,0.543596,0.0,-3.666936,-0.959618,0.0,3.790420,0.379857,0.626743,0.0,0.732870,145703,4648,CF,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H18,0 days 00:52:45.120000,9,1.44,2.0,0.0,0.0,0.0,67.758713,-0.008023,0.999968,20.688536,-0.271826,-0.962346,0.939633,-0.342183,2.0,2.0,3.298762
3,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-18.170985,-6.893900,0.0,-3.449665,-0.014393,0.0,3.449695,-0.142334,1.727880,0.0,1.733733,161110,316,CM,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A05,0 days 00:52:45.120000,9,1.44,2.0,0.0,1.0,0.0,71.006436,0.097088,0.995276,17.091603,0.106123,-0.994353,0.995255,-0.097302,2.0,2.0,3.298762
4,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-19.764015,6.894847,0.0,-4.435351,-0.583185,0.0,4.473527,-0.550788,1.268120,0.0,1.382569,187326,316,CAM,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A07,0 days 00:52:45.120000,9,1.44,2.0,0.0,1.0,0.0,72.592196,-0.094981,0.995479,19.509544,-0.613799,-0.789463,0.975012,-0.222151,2.0,2.0,3.298762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-32.826194,0.422628,0.0,3.326097,-0.179265,0.0,3.330925,0.623070,0.943717,0.0,1.130848,500140,4648,LB,4648,False,"[[8.34407528282654, 2.908493628814181, 8.62360...","[[1.0458654020964481e-12, 0.003411566431691531...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H04,NaT,1,NaN,NaN,NaN,1.0,0.0,85.327241,-0.004953,0.999988,32.101536,-0.808083,0.589068,0.563420,-0.826171,NaN,1.0,8.885287
226,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-24.015056,-2.508657,0.0,4.643271,0.268948,0.0,4.651053,0.619575,-0.237683,0.0,0.663601,500142,4648,CM,4648,False,"[[8.34407528282654, 2.908493628814181, 8.62360...

In [ ]:

def time_since_last_opponent_action(merged_df):
    """
    각 이벤트 시점까지의 가장 최근 상대 팀 액션 이후 경과 시간을 계산합니다
    action_id, time_seconds 및 계산된 피처를 포함하는 DataFrame을 반환합니다.

    Args:
        merged_df (pd.DataFrame): 'action_id', 'time_seconds' 컬럼을 포함하는 DataFrame.

    Returns:
        pd.DataFrame: 'action_id', 'time_seconds', 'elapsed_time' 컬럼을 담은 DataFrame.
    """
    result = []

    for idx, row in merged_df.iterrows():
        current_time = row['time_seconds']
        current_team = row['tID']

        # 상대 팀 이벤트 중 현재 시간보다 이전 것만 필터링
        opponent_events = merged_df[(merged_df['tID'] != current_team) &
                                 (merged_df['time_seconds'] < current_time)]

        if not opponent_events.empty:
            last_opponent_time = opponent_events['time_seconds'].max()
            time_diff = current_time - last_opponent_time
        else:
            time_diff = 0.0

    return pd.DataFrame({
        'action_id': merged_df['action_id'],
        'time_seconds': merged_df['time_seconds'],
        'time_since_last_opponent_action': time_diff
    }, index=merged_df.index)


,game_id,period_id,timestamp,frame_id,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows,player_code
1915786,126285,2.0,0 days 00:52:44.480000,146612,alive,316,29.767707,-2.009037,0.0,4.386689,...,139210,316,LB,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A01
1915787,126285,2.0,0 days 00:52:44.480000,146612,alive,316,11.954909,8.306194,0.0,0.114114,...,143410,316,CM,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A06
1915788,126285,2.0,0 days 00:52:44.480000,146612,alive,316,15.256533,-0.543596,0.0,3.666936,...,145703,4648,CF,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H18
1915789,126285,2.0,0 days 00:52:44.480000,146612,alive,316,18.170985,6.893900,0.0,3.449665,...,161110,316,CM,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A05
1915790,126285,2.0,0 days 00:52:44.480000,146612,alive,316,19.764015,-6.894847,0.0,4.435351,...,187326,316,CAM,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918679,126285,2.0,0 days 00:52:49.480000,146737,alive,4648,32.826194,-0.422628,0.0,-3.326097,...,500140,4648,LB,4648,False,"[[8.34407528282654, 2.908493628814181, 8.62360...","[[1.0458654020964481e-12, 0.003411566431691531...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H04
1918680,126285,2.0,0 days 00:52:49.480000,146737,alive,4648,24.015056,2.508657,0.0,-4.643271,...,500142,4648,CM,4648,False,"[[8.34407528282654, 2.908493628814181, 8.62360...","[[1.0458654020964481e-12, 0.003411566431691531...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H17
1918681,126285,2.0,0 days 00:52:49.480000,146737,alive,4648,34.208471,18.352432,0.0,-3.135064,...,62386,4648,RB,4648,False,"[[8.34407528282654, 2.908493628814181, 8.62360...","[[1.0458654020964481e-12, 0.003411566431691531...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H03
1918682,126285,2.0,0 days 00:52:49.480000,146737,alive,4648,3.431331,22.009390,0.0,2.074155,...,77579,316,CB,4648,False,"[[8.34407528282654, 2.908493628814181, 8.62360...","[[1.0458654020964481e-12, 0.003411566431691531...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A02


In [ ]:
X_slice.to_csv(f"{TEST_MATCH_ID}_X_slice.csv", index=False)

In [ ]:
X_slice.columns

Index(['game_id', 'period_id', 'timestamp', 'frame_id', 'ball_state',
       'ori_ball_owning_team_id', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'v', 'ax',
       'ay', 'az', 'a', 'id', 'team_id', 'position_name',
       'ball_owning_team_id', 'is_ball_carrier', 'time_to_intercept',
       'probability_to_intercept', 'columns', 'rows', 'player_code'],
      dtype='object')

In [ ]:
def add_elapsed_time(df):
    """
    각 프레임의 경기 전체 기준 경과 시간을 계산하여 'elapsed_time' 컬럼을 추가합니다.
    전반전은 그대로, 후반전은 2700초를 더해 누적 시간으로 변환합니다.

    Args:
        df (pd.DataFrame): 'timestamp', 'period_id' 컬럼을 포함한 X_slice

    Returns:
        pd.DataFrame: 'elapsed_time' 컬럼이 추가된 DataFrame
    """
    df = df.copy()
    # timedelta → 초로 변환
    df['elapsed_time'] = df['timestamp'].dt.total_seconds()
    
    # 후반전(period 2)은 2700초(=45분)를 더함
    df.loc[df['period_id'] == 2, 'elapsed_time'] += 2700
    
    return df

In [ ]:
add_elapsed_time(X_slice)[['id','period_id','timestamp','elapsed_time']].head(50)

,id,period_id,timestamp,elapsed_time
1915786,139210,2.0,0 days 00:52:44.480000,5864.48
1915787,143410,2.0,0 days 00:52:44.480000,5864.48
1915788,145703,2.0,0 days 00:52:44.480000,5864.48
1915789,161110,2.0,0 days 00:52:44.480000,5864.48
1915790,187326,2.0,0 days 00:52:44.480000,5864.48
1915791,188178,2.0,0 days 00:52:44.480000,5864.48
1915792,188267,2.0,0 days 00:52:44.480000,5864.48
1915793,250079,2.0,0 days 00:52:44.480000,5864.48
1915794,250101,2.0,0 days 00:52:44.480000,5864.48
1915795,286696,2.0,0 days 00:52:44.480000,5864.48


In [ ]:
pd.set_option('display.max_columns', None)
all_features_seqs[0].shape

IndexError: list index out of range

In [ ]:
X_slice

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows,player_code,time_seconds,type_id,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle,distance_ball_to_goal
2,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-15.256533,0.543596,0.0,-3.666936,-0.959618,0.0,3.790420,0.379857,0.626743,0.0,0.732870,145703,4648,CF,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H18,0 days 00:52:45.120000,9,0.0,0.0,67.758713,-0.008023,0.999968,20.688536,-0.271826,-0.962346,0.939633,-0.342183,87.813139
5,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-17.146921,16.611762,0.0,-5.532716,-2.093201,0.0,5.915440,-1.054783,-0.316908,0.0,1.101362,188178,4648,LM,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H14,0 days 00:52:45.120000,9,0.0,0.0,71.600588,-0.232006,0.972714,28.199751,-0.769221,-0.638983,0.898273,-0.439438,87.813139
7,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-43.652008,-0.938508,0.0,-0.584257,1.352173,0.0,1.473000,-1.002972,3.503382,0.0,3.644124,250079,4648,GK,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H00,0 days 00:52:45.120000,9,0.0,1.0,96.156588,0.009760,0.999952,9.442656,-0.438603,0.898681,0.480428,0.877034,87.813139
8,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-24.463659,-12.946000,0.0,-3.684163,-0.247245,0.0,3.692450,0.881502,3.964805,0.0,4.061616,250101,4648,CM,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H06,0 days 00:52:45.120000,9,0.0,0.0,78.044883,0.165879,0.986146,13.282104,0.592219,-0.805777,0.987180,-0.159609,87.813139
13,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-10.359025,0.009512,0.0,-2.616415,-1.522029,0.0,3.026913,0.206902,0.914455,0.0,0.937570,369783,4648,CF,316,False,"[[5.695313464174637, 1.5716809266522551, 2.860...","[[4.530247118607717e-08, 0.0, 0.00413550747774...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H19,0 days 00:52:45.120000,9,0.0,0.0,62.859026,-0.000151,1.000000,25.323774,-0.200981,-0.979595,0.813787,-0.581163,87.813139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,38.314007,-3.147688,0.0,0.685428,-0.989609,0.0,1.203802,0.198598,0.531478,0.0,0.567371,344466,316,GK,4648,False,"[[8.34407528282654, 2.908493628814181, 8.62360...","[[1.0458654020964481e-12, 0.003411566431691531...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A00,NaT,1,0.0,1.0,14.531013,0.216619,0.976256,56.819258,-0.393711,-0.919234,0.998135,-0.061046,71.149721
219,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-4.277243,-0.273143,0.0,4.231130,-3.082920,0.0,5.235155,-0.289021,-1.427191,0.0,1.456162,344467,316,CB,4648,False,"[[8.34407528282654, 2.908493628814181, 8.62360...","[[1.0458654020964481e-12, 0.003411566431691531...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A03,NaT,1,0.0,0.0,56.777900,0.004811,0

In [ ]:
X_slice

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle,distance_ball_to_goal
2,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-15.256533,0.543596,0.0,-3.666936,...,0.0,67.758713,-0.008023,0.999968,20.688536,-0.271826,-0.962346,0.939633,-0.342183,87.813139
5,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-17.146921,16.611762,0.0,-5.532716,...,0.0,71.600588,-0.232006,0.972714,28.199751,-0.769221,-0.638983,0.898273,-0.439438,87.813139
7,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-43.652008,-0.938508,0.0,-0.584257,...,1.0,96.156588,0.009760,0.999952,9.442656,-0.438603,0.898681,0.480428,0.877034,87.813139
8,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-24.463659,-12.946000,0.0,-3.684163,...,0.0,78.044883,0.165879,0.986146,13.282104,0.592219,-0.805777,0.987180,-0.159609,87.813139
13,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-10.359025,0.009512,0.0,-2.616415,...,0.0,62.859026,-0.000151,1.000000,25.323774,-0.200981,-0.979595,0.813787,-0.581163,87.813139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,38.314007,-3.147688,0.0,0.685428,...,1.0,14.531013,0.216619,0.976256,56.819258,-0.393711,-0.919234,0.998135,-0.061046,71.149721
219,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-4.277243,-0.273143,0.0,4.231130,...,0.0,56.777900,0.004811,0.999988,27.022513,-0.934219,-0.356701,0.922442,-0.386136,71.149721
221,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-28.706064,12.727769,0.0,1.239774,...,0.0,82.197451,-0.154844,0.987939,41.005920,-0.932691,0.360677,-0.240256,-0.970710,71.149721
228,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-3.431331,-22.009390,0.0,-2.074155,...,0.0,60.105965,0.366176,0.930545,11.056365,-0.317346,-0.948310,0.662319,0.749222,71.149721


In [ ]:
X_slice

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle,distance_ball_to_goal
2,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-15.256533,0.543596,0.0,-3.666936,...,0.0,67.758713,-0.008023,0.999968,20.688536,-0.271826,-0.962346,0.939633,-0.342183,87.813139
5,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-17.146921,16.611762,0.0,-5.532716,...,0.0,71.600588,-0.232006,0.972714,28.199751,-0.769221,-0.638983,0.898273,-0.439438,87.813139
7,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-43.652008,-0.938508,0.0,-0.584257,...,1.0,96.156588,0.009760,0.999952,9.442656,-0.438603,0.898681,0.480428,0.877034,87.813139
8,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-24.463659,-12.946000,0.0,-3.684163,...,0.0,78.044883,0.165879,0.986146,13.282104,0.592219,-0.805777,0.987180,-0.159609,87.813139
13,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-10.359025,0.009512,0.0,-2.616415,...,0.0,62.859026,-0.000151,1.000000,25.323774,-0.200981,-0.979595,0.813787,-0.581163,87.813139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,38.314007,-3.147688,0.0,0.685428,...,1.0,14.531013,0.216619,0.976256,56.819258,-0.393711,-0.919234,0.998135,-0.061046,71.149721
219,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-4.277243,-0.273143,0.0,4.231130,...,0.0,56.777900,0.004811,0.999988,27.022513,-0.934219,-0.356701,0.922442,-0.386136,71.149721
221,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-28.706064,12.727769,0.0,1.239774,...,0.0,82.197451,-0.154844,0.987939,41.005920,-0.932691,0.360677,-0.240256,-0.970710,71.149721
228,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-3.431331,-22.009390,0.0,-2.074155,...,0.0,60.105965,0.366176,0.930545,11.056365,-0.317346,-0.948310,0.662319,0.749222,71.149721


In [ ]:
X_slice_test

NameError: name 'X_slice_test' is not defined

In [ ]:
X_slice

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle
2,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-15.256533,0.543596,0.0,-3.666936,...,0.0,0.0,67.758713,-0.008023,0.999968,20.688536,-0.271826,-0.962346,0.939633,-0.342183
5,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-17.146921,16.611762,0.0,-5.532716,...,0.0,0.0,71.600588,-0.232006,0.972714,28.199751,-0.769221,-0.638983,0.898273,-0.439438
7,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-43.652008,-0.938508,0.0,-0.584257,...,0.0,1.0,96.156588,0.009760,0.999952,9.442656,-0.438603,0.898681,0.480428,0.877034
8,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-24.463659,-12.946000,0.0,-3.684163,...,0.0,0.0,78.044883,0.165879,0.986146,13.282104,0.592219,-0.805777,0.987180,-0.159609
13,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-10.359025,0.009512,0.0,-2.616415,...,0.0,0.0,62.859026,-0.000151,1.000000,25.323774,-0.200981,-0.979595,0.813787,-0.581163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,38.314007,-3.147688,0.0,0.685428,...,0.0,1.0,14.531013,0.216619,0.976256,56.819258,-0.393711,-0.919234,0.998135,-0.061046
219,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-4.277243,-0.273143,0.0,4.231130,...,0.0,0.0,56.777900,0.004811,0.999988,27.022513,-0.934219,-0.356701,0.922442,-0.386136
221,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-28.706064,12.727769,0.0,1.239774,...,0.0,0.0,82.197451,-0.154844,0.987939,41.005920,-0.932691,0.360677,-0.240256,-0.970710
228,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-3.431331,-22.009390,0.0,-2.074155,...,0.0,0.0,60.105965,0.366176,0.930545,11.056365,-0.317346,-0.948310,0.662319,0.749222


In [ ]:
X_slice_test

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle,distance_ball_goal
0,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-29.767707,2.009037,0.0,-4.386689,...,0.0,82.292234,-0.024413,0.999702,8.910554,-0.795587,-0.605839,0.920331,-0.391141,87.813139
1,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-11.954909,-8.306194,0.0,-0.114114,...,0.0,64.987908,0.127811,0.991798,23.434288,0.137666,-0.990479,0.662853,0.748749,87.813139
2,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-15.256533,0.543596,0.0,-3.666936,...,0.0,67.758713,-0.008023,0.999968,20.688536,-0.271826,-0.962346,0.939633,-0.342183,87.813139
3,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-18.170985,-6.893900,0.0,-3.449665,...,0.0,71.006436,0.097088,0.995276,17.091603,0.106123,-0.994353,0.995255,-0.097302,87.813139
4,146612,126285,2.0,0 days 00:52:44.480000,alive,316,-19.764015,6.894847,0.0,-4.435351,...,0.0,72.592196,-0.094981,0.995479,19.509544,-0.613799,-0.789463,0.975012,-0.222151,87.813139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-32.826194,0.422628,0.0,3.326097,...,0.0,85.327241,-0.004953,0.999988,32.101536,-0.808083,0.589068,0.563420,-0.826171,71.149721
226,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-24.015056,-2.508657,0.0,4.643271,...,0.0,76.556170,0.032769,0.999463,25.128086,-0.915686,0.401896,0.467815,-0.883826,71.149721
227,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-34.208471,-18.352432,0.0,3.135064,...,0.0,88.629401,0.207069,0.978326,21.520297,-0.332972,0.942937,0.881544,-0.472102,71.149721
228,146737,126285,2.0,0 days 00:52:49.480000,alive,4648,-3.431331,-22.009390,0.0,-2.074155,...,0.0,60.105965,0.366176,0.930545,11.056365,-0.317346,-0.948310,0.662319,0.749222,71.149721


In [ ]:
feature_cols = _infer_feature_cols()
#X_slice.loc[:, 'id'] = pd.Categorical(X_slice['id'], categories=agents_order, ordered=True)
X_slice_test['id'] = pd.Categorical(X_slice_test['id'], categories=agents_order, ordered=True)

# Sort the players by their ID to maintain a consistent order
X_slice_test = X_slice_test.sort_values(by=['frame_id', 'id'])

# Get the features
x_tensor = torch.tensor(X_slice_test[feature_cols].values, dtype=torch.float32)

X_slice_pressing = X_slice_test[X_slice_test['is_ball_carrier']==True]['probability_to_intercept']
X_slice_pressing = X_slice_test.dropna()
pressing_intensity_tensor = torch.tensor(np.stack(X_slice_pressing.map(lambda x: np.stack(x)).values), dtype=torch.float32)
_, h, w = pressing_intensity_tensor.shape
pad_h = NUM_TEAM_PLAYERS - h
pad_w = NUM_TEAM_PLAYERS - w
pressing_intensity_tensor = F.pad(pressing_intensity_tensor, (0, pad_w, 0, pad_h), "constant", 0)

x_tensor = x_tensor.reshape(-1, NUM_AGENTS, len(feature_cols))
y_tensor = torch.tensor(label, dtype=torch.long)

# Debug 
if x_tensor.isnan().any():
    print("Nan found", TEST_MATCH_ID, period_id, frame_id)
match_info = f"{TEST_MATCH_ID}-{period_id}-{frame_id}"
all_features_seqs = []
all_pressintensity_seqs = []
all_labels = []
all_pressed_ids = []
all_presser_ids = []
all_agent_orders = []
all_match_infos = []
all_features_seqs.append(x_tensor)
all_pressintensity_seqs.append(pressing_intensity_tensor)
all_labels.append(y_tensor)
all_pressed_ids.append(pressed_player)
all_presser_ids.append(pressing_player)    
all_agent_orders.append(agents_order)
all_match_infos.append(match_info)

TypeError: arrays to stack must be passed as a "sequence" type such as list or tuple.

In [ ]:
agents_order


['145703',
 '188178',
 '250079',
 '250101',
 '369783',
 '500133',
 '500135',
 '500138',
 '500140',
 '500142',
 '62386',
 '139210',
 '143410',
 '161110',
 '187326',
 '188267',
 '286696',
 '344465',
 '344466',
 '344467',
 '500124',
 '77579',
 'ball']

In [ ]:
total_df.columns

Index(['game_id', 'period_id', 'timestamp', 'frame_id', 'ball_state',
       'ori_ball_owning_team_id', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'v', 'ax',
       'ay', 'az', 'a', 'id', 'team_id', 'position_name',
       'ball_owning_team_id', 'is_ball_carrier', 'time_to_intercept',
       'probability_to_intercept', 'columns', 'rows', 'player_code'],
      dtype='object')

In [ ]:
total_df

,game_id,period_id,timestamp,frame_id,ball_state,ori_ball_owning_team_id,x,y,z,vx,...,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows,player_code
0,126285,1.0,0 days 00:00:01.560000,39,alive,4648,12.640970,-19.940540,0.0,0.244768,...,139210,316,LB,4648,False,"[[6.670151156799039, 5.83531814870236, 5.99660...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188266, 24889...","[250079, 250101, 250102, 500133, 500135, 50013...",A01
1,126285,1.0,0 days 00:00:01.560000,39,alive,4648,4.944908,-3.895127,0.0,-0.811747,...,143410,316,CM,4648,False,"[[6.670151156799039, 5.83531814870236, 5.99660...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188266, 24889...","[250079, 250101, 250102, 500133, 500135, 50013...",A06
2,126285,1.0,0 days 00:00:01.560000,39,alive,4648,6.581582,3.584146,0.0,-1.557800,...,161110,316,CM,4648,False,"[[6.670151156799039, 5.83531814870236, 5.99660...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188266, 24889...","[250079, 250101, 250102, 500133, 500135, 50013...",A05
3,126285,1.0,0 days 00:00:01.560000,39,alive,4648,-2.128864,9.077650,0.0,-0.702102,...,187326,316,CAM,4648,False,"[[6.670151156799039, 5.83531814870236, 5.99660...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188266, 24889...","[250079, 250101, 250102, 500133, 500135, 50013...",A07
4,126285,1.0,0 days 00:00:01.560000,39,alive,4648,-2.077961,18.140258,0.0,-1.111589,...,188266,316,RW,4648,False,"[[6.670151156799039, 5.83531814870236, 5.99660...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188266, 24889...","[250079, 250101, 250102, 500133, 500135, 50013...",A10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1921048,126285,2.0,0 days 00:52:53.600000,146840,alive,4648,24.796400,-0.461400,0.0,-0.204866,...,500140,4648,LB,4648,False,"[[4.056718478588534, 1.9920179718405002, 3.307...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H04
1921049,126285,2.0,0 days 00:52:53.600000,146840,alive,4648,17.598900,8.309600,0.0,0.828691,...,500142,4648,CM,4648,False,"[[4.056718478588534, 1.9920179718405002, 3.307...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H17
1921050,126285,2.0,0 days 00:52:53.600000,146840,alive,4648,26.302700,28.339200,0.0,-1.623980,...,62386,4648,RB,4648,False,"[[4.056718478588534, 1.9920179718405002, 3.307...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",H03
1921051,126285,2.0,0 days 00:52:53.600000,146840,alive,4648,7.605300,33.147500,0.0,-0.300878,...,77579,316,CB,4648,False,"[[4.056718478588534, 1.9920179718405002, 3.307...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[139210, 143410, 161110, 187326, 188267, 28669...","[145703, 188178, 250079, 250101, 369783, 50013...",A02
